In [1]:
#네이버 영화 리뷰 감성 분석
#6가지 실험 
# 1_1 LSTM 모델 random 임배딩 사용
# 1_2 LSTM 모델 word2vec 임배딩 사용
# 2_1 cnn 모델 random 임배딩 사용
# 2_2 cnn 모델 word2vec 임배딩 사용
# 3_1 LSTM+cnn 모델 random 임배딩 사용
# 3_2 LSTM+cnn 모델 word2vec 임배딩 사용

#6가지의 임배딩 파일 비교 분석 

In [5]:
#!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
#!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
#!mv ratings_*.txt ~/work/workplace/AIFFEL_quest_rs/Exploration/Quest05/sentiment_classification/data

In [1]:
!pip install konlpy
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab/
!bash install_mecab-ko_on_colab_light_220429.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 138 (delta 26), reused 22 (delta 8), pack-reused 91 (from 1)
Receiving objects: 100% (138/138), 1.72 MiB | 6.16 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/home/jovyan/work/workplace/AIFFEL_quest_rs/Exploration/Quest05/Mecab-ko-for-Google-Colab
install_mecab-ko_on_colab_light_220429.sh: line 4: cd: /content: No such file or directory
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2025-08-05 05:34:39--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.143.22, 104.192.143.21, 104.192.143.23, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.143.22|:443... connected.
HTTP request sent, awaiting respon

In [1]:
! python --version

Python 3.12.11


In [2]:
# 의존성 연결을 위해 다운그레이드를 진행합니다.
!pip install scipy==1.12.0 numpy==1.26.3

In [3]:
# 데이터 준비 및 확인

import pandas as pd
import os

# train_data = pd.read_table(os.getenv("HOME") + '/work/sentiment_classification/data/ratings_train.txt')
# test_data = pd.read_table(os.getenv("HOME") + '/work/sentiment_classification/data/ratings_test.txt')

# train_data.head()


data_path = os.path.join(os.getenv("HOME"), 'work', 'workplace', 'AIFFEL_quest_rs', 'Exploration', 'Quest05', 'sentiment_classification', 'data')

train_data = pd.read_table(os.path.join(data_path, 'ratings_train.txt'))
test_data = pd.read_table(os.path.join(data_path, 'ratings_test.txt'))

train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [9]:
#데이터 전처리
 #데이터 중복 제거
 #데이터 Non값 제거
 #한국어 토크나이저로 토큰화
 #불용어 제거 

from konlpy.tag import Mecab
import numpy as np
from collections import Counter

try:
    # MeCab 설치 스크립트가 설치한 mecabrc 설정 파일의 경로를 환경 변수에 설정
    # 이 파일은 일반적으로 설치 스크립트가 다운로드한 소스 폴더 안에 있습니다.
    os.environ['MECABRC'] = os.path.join(os.getenv("HOME"), 'work', 'workplace', 'AIFFEL_quest_rs', 'Exploration', 'Quest05', 'Mecab-ko-for-Google-Colab', 'mecab-0.996-ko-0.9.2', 'etc', 'mecabrc')
    
    # MeCab 사전의 실제 경로도 고객님께서 알려주신 설치 경로로 재지정합니다.
    # 설치 스크립트가 'Mecab-ko-for-Google-Colab' 폴더 안에 사전을 생성한 것으로 추정합니다.
    base_path = os.path.join(os.getenv("HOME"), 'work', 'workplace', 'AIFFEL_quest_rs', 'Exploration', 'Quest05', 'Mecab-ko-for-Google-Colab')
    mecab_dic_path = os.path.join(base_path, 'mecab-ko-dic')
    
    # 구성된 경로를 사용하여 Mecab 객체를 생성합니다.
    tokenizer = Mecab(dicpath=mecab_dic_path)
    print("Mecab 토크나이저가 성공적으로 초기화되었습니다.")
except Exception as e:
    # 경로가 올바르지 않으면 오류를 출력합니다.
    raise Exception(f"MeCab dictionary not found or mecabrc file is missing. Please check your installation. The dictionary path attempted was: {mecab_dic_path}")

# tokenizer = Mecab()
#tokenizer = Mecab(dicpath='/usr/local/lib/mecab/dic/mecab-ko-dic')
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

#데이터 클리닝
def load_data(train_data, test_data, num_words=10000):
    train_data.drop_duplicates(subset=['document'], inplace=True)
    train_data = train_data.dropna(how = 'any')
    test_data.drop_duplicates(subset=['document'], inplace=True)
    test_data = test_data.dropna(how = 'any')

#토큰화 불용 제거 
    X_train = []
    for sentence in train_data['document']:
        temp_X = tokenizer.morphs(sentence) # 토큰화
        temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
        X_train.append(temp_X)

    X_test = []
    for sentence in test_data['document']:
        temp_X = tokenizer.morphs(sentence) # 토큰화
        temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
        X_test.append(temp_X)

    # vocab 구
    words = np.concatenate(X_train).tolist()
    counter = Counter(words)
    counter = counter.most_common(10000-4)
    vocab = ['', '', '', ''] + [key for key, _ in counter]
    word_to_index = {word:index for index, word in enumerate(vocab)}

    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index[''] for word in wordlist]

    X_train = list(map(wordlist_to_indexlist, X_train))
    X_test = list(map(wordlist_to_indexlist, X_test))

    return X_train, np.array(list(train_data['label'])), X_test, np.array(list(test_data['label'])), word_to_index

X_train, y_train, X_test, y_test, word_to_index = load_data(train_data, test_data)

Exception: MeCab dictionary not found or mecabrc file is missing. Please check your installation. The dictionary path attempted was: /home/jovyan/work/workplace/AIFFEL_quest_rs/Exploration/Quest05/Mecab-ko-for-Google-Colab/mecab-ko-dic

In [ ]:
index_to_word = {index:word for word, index in word_to_index.items()}

In [ ]:
# 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트 벡터로 변환해 주는 함수입니다.
# 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다.
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']]+[word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in sentence.split()]

# 여러 개의 문장 리스트를 한꺼번에 단어 인덱스 리스트 벡터로 encode해 주는 함수입니다.
def get_encoded_sentences(sentences, word_to_index):
    return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다.
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

# 여러 개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다.
def get_decoded_sentences(encoded_sentences, index_to_word):
    return [get_decoded_sentence(encoded_sentence, index_to_word) for encoded_sentence in encoded_sentences]

In [ ]:
# 함수가 정상 작동하는지 확인하기 위한 테스트 코드

# 테스트를 위한 가상의 word_to_index와 index_to_word 딕셔너리를 만듭니다.
# 실제 데이터의 딕셔너리와 구조가 유사해야 합니다.
test_word_to_index = {
    '<PAD>': 0, '<BOS>': 1, '<UNK>': 2, '': 3,
    'hello': 4, 'world': 5, 'python': 6, 'is': 7, 'great': 8
}
test_index_to_word = {index: word for word, index in test_word_to_index.items()}

# 테스트 문장
test_sentence = 'hello world python is great'
test_sentences = ['hello world', 'python is great', 'hello python']

# 1. get_encoded_sentence 테스트
encoded_sentence = get_encoded_sentence(test_sentence, test_word_to_index)
print(f"원본 문장: {test_sentence}")
print(f"인코딩 결과: {encoded_sentence}")
print("-" * 30)

# 2. get_encoded_sentences 테스트
encoded_sentences = get_encoded_sentences(test_sentences, test_word_to_index)
print(f"원본 문장 리스트: {test_sentences}")
print(f"인코딩 결과 리스트: {encoded_sentences}")
print("-" * 30)

# 3. get_decoded_sentence 테스트
decoded_sentence = get_decoded_sentence(encoded_sentence, test_index_to_word)
print(f"인코딩된 문장: {encoded_sentence}")
print(f"디코딩 결과: {decoded_sentence}")
print("-" * 30)

# 4. get_decoded_sentences 테스트
decoded_sentences = get_decoded_sentences(encoded_sentences, test_index_to_word)
print(f"인코딩된 문장 리스트: {encoded_sentences}")
print(f"디코딩 결과 리스트: {decoded_sentences}")

In [ ]:
# 3) 모델 구성을 위한 데이터 분석 및 가공

In [ ]:
 #데이터셋 내 문장 길이 분포
 #적절한 최대 문장 길이 지정


total_data_text = list(X_train) + list(X_test)
# 텍스트데이터 문장길이의 리스트를 생성한 후
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다.
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 예를들어, 최대 길이를 (평균 + 2*표준편차)로 한다면,
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print(
    f'''전체 문장의 
    {np.sum(num_tokens < max_tokens) / len(num_tokens)}%가 
    maxlen 설정값 이내에 포함됩니다. ''')

In [ ]:
# 문장 길이 통일(패딩추가) 및 텐서 변환
import torch  # 딥러닝 프레임워크 PyTorch 라이브러리

def pad_sequences(data, maxlen):
    padded_data = []
    for sentence in data:
        if len(sentence) < maxlen:
            # 문장 길이가 maxlen보다 짧으면, 0(패딩)을 추가하여 길이 맞추기
            sentence = sentence + [0] * (maxlen - len(sentence))
        else:
            # 문장 길이가 maxlen보다 길면, 뒷부분을 잘라서 길이 맞추기
            sentence = sentence[:maxlen]
        padded_data.append(sentence)
    return np.array(padded_data)  # 결과를 numpy 배열로 반환

X_train_padded = pad_sequences(X_train, maxlen)  # 훈련 데이터의 길이를 maxlen에 맞춤
X_test_padded = pad_sequences(X_test, maxlen)  # 테스트 데이터의 길이를 maxlen에 맞춤

X_train_tensor = torch.tensor(X_train_padded, dtype=torch.long)  # 패딩된 훈련 데이터를 PyTorch 텐서로 변환
X_test_tensor = torch.tensor(X_test_padded, dtype=torch.long)  # 패딩된 테스트 데이터를 PyTorch 텐서로 변환

print(X_train_tensor.shape)  # 변환된 훈련 텐서의 형태(shape) 출력

In [ ]:
# 4)모델 구성 및 validation set 구성

In [ ]:
# 1_1 LSTM 기반 감성 분석 모델 정의 및 구조 출력
import torch  # 딥러닝 프레임워크 PyTorch 라이브러리
import torch.nn as nn  # 신경망 모듈
import torch.nn.functional as F  # 신경망 함수 모음

vocab_size = 10000  # 모델이 다룰 수 있는 단어의 총 개수
word_vector_dim = 16  # 각 단어를 표현하는 워드 벡터의 차원

# 모델 설계: nn.Module을 상속받아 SentimentModel 클래스 정의
class SentimentModel(nn.Module):
    # 모델의 레이어(Layer)들을 정의
    def __init__(self, vocab_size, word_vector_dim):
        super(SentimentModel, self).__init__()  # nn.Module의 생성자 호출
        self.embedding = nn.Embedding(vocab_size, word_vector_dim)  # 단어를 워드 벡터로 변환하는 임베딩 레이어
        self.lstm = nn.LSTM(word_vector_dim, 8, batch_first=True)  # 워드 벡터 시퀀스를 처리하는 LSTM 레이어
        self.fc1 = nn.Linear(8, 8)  # LSTM의 최종 출력을 받아서 처리하는 첫 번째 완전 연결 레이어
        self.fc2 = nn.Linear(8, 1)  # 최종 출력을 0과 1 사이의 값으로 만드는 두 번째 완전 연결 레이어

    # 모델의 순전파(forward) 로직을 정의
    def forward(self, x):
        x = self.embedding(x)  # 입력(단어 인덱스)을 임베딩 레이어에 통과시켜 워드 벡터로 변환
        x, (hn, cn) = self.lstm(x)  # 임베딩 벡터 시퀀스를 LSTM에 통과
        x = x[:, -1, :]  # LSTM의 최종 시점(sequence의 마지막)의 은닉 상태를 가져옴
        x = F.relu(self.fc1(x))  # 첫 번째 완전 연결 레이어에 통과시키고 ReLU 활성화 함수 적용
        x = torch.sigmoid(self.fc2(x))  # 두 번째 완전 연결 레이어에 통과시키고 시그모이드 활성화 함수 적용 (0~1 사이 값)
        return x  # 최종 출력값 반환

model_lstm_random = SentimentModel(vocab_size, word_vector_dim)  # 정의된 클래스를 바탕으로 모델 객체 생성
print(model_lstm_random)  # 모델의 레이어 구조와 파라미터 정보 출력

In [ ]:
# validation set 10000건 분리
X_val = X_train_tensor[:10000]
y_val = y_train[:10000]

# validation set을 제외한 나머지 15000건
partial_X_train = X_train_tensor[10000:]
partial_y_train = y_train[10000:]

print(partial_X_train.shape)
print(partial_y_train.shape)

In [ ]:
# 5) 모델 훈련 개

In [ ]:
# 1-1 LSTM을 이용한 감성 분석 모델 훈련
import torch.optim as optim  # 최적화 도구 모음
import torch.nn.functional as F  # 활성화 함수, 손실 함수 등
from torch.utils.data import DataLoader, TensorDataset  # 데이터 로딩 유틸리티

optimizer = optim.Adam(model_lstm_random.parameters())  # 모델의 파라미터를 업데이트할 Adam 최적화 도구 설정
loss_fn = torch.nn.BCELoss()  # 이진 분류에 적합한 Binary Cross Entropy 손실 함수 설정

partial_X_train_tensor = partial_X_train.detach().clone().to(torch.long)  # 훈련용 입력 텐서를 복사하고 데이터 타입을 long으로 변환
partial_y_train_tensor = torch.tensor(partial_y_train, dtype=torch.float)  # 훈련용 라벨 데이터를 PyTorch 텐서로 변환 (BCELoss를 위해 float 타입 사용)

X_val_tensor = X_val.detach().clone().to(torch.long)  # 검증용 입력 텐서를 복사하고 데이터 타입을 long으로 변환
y_val_tensor = torch.tensor(y_val, dtype=torch.float)  # 검증용 라벨 데이터를 PyTorch 텐서로 변환

train_dataset = TensorDataset(partial_X_train_tensor, partial_y_train_tensor)  # 훈련 데이터와 라벨을 묶어 데이터셋 생성
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)  # 검증 데이터와 라벨을 묶어 데이터셋 생성

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)  # 훈련용 데이터로더 생성 (배치 크기 512, 데이터 섞기)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)  # 검증용 데이터로더 생성 (배치 크기 512, 데이터 섞지 않음)

epochs = 30  # 훈련 반복 횟수 설정
train_losses = []  # 에포크별 훈련 손실을 저장할 리스트
val_losses = []  # 에포크별 검증 손실을 저장할 리스트
train_accs = []  # 에포크별 훈련 정확도를 저장할 리스트
val_accs = []  # 에포크별 검증 정확도를 저장할 리스트

# 조기 종료 관련 하이퍼파라미터
patience = 5          # 검증 손실이 5번 연속으로 개선되지 않으면 훈련 중단
min_delta = 0.0001    # 최소한 이 값 이상 손실이 감소해야 개선으로 인정

best_loss = np.Inf    # 초기 최저 손실값 설정
patience_counter = 0  # 손실 개선 실패 횟수 카운터


for epoch in range(epochs):
    # 훈련 단계 시작
    model_lstm_random.train()  # 모델을 훈련 모드로 설정
    running_loss = 0.0  # 현재 에포크의 누적 손실 초기화
    correct = 0  # 현재 에포크의 정답 수 초기화
    total = 0  # 현재 에포크의 전체 데이터 수 초기화

    for inputs, labels in train_loader:
        optimizer.zero_grad()  # 이전 스텝에서 계산된 기울기 초기화

        outputs = model_lstm_random(inputs)  # 모델에 입력 데이터를 넣어 예측값 계산
        loss = loss_fn(outputs.squeeze(), labels)  # 예측값과 실제 라벨 간의 손실 계산
        loss.backward()  # 손실에 대한 기울기 역전파
        optimizer.step()  # 모델의 파라미터 업데이트

        running_loss += loss.item()  # 현재 배치 손실을 누적
        predicted = (outputs.squeeze() > 0.5).float()  # 예측값이 0.5보다 크면 긍정(1), 아니면 부정(0)으로 판단
        correct += (predicted == labels).sum().item()  # 맞춘 개수 누적
        total += labels.size(0)  # 전체 데이터 수 누적

    train_losses.append(running_loss / len(train_loader))  # 에포크별 평균 훈련 손실 기록
    train_accs.append(correct / total)  # 에포크별 훈련 정확도 기록

    # 검증 단계 시작
    model_lstm_random.eval()  # 모델을 평가 모드로 설정
    val_loss = 0.0  # 현재 에포크의 누적 검증 손실 초기화
    val_correct = 0  # 현재 에포크의 정답 수 초기화
    val_total = 0  # 현재 에포크의 전체 데이터 수 초기화

    with torch.no_grad():  # 기울기 계산을 비활성화 (메모리 절약, 연산 속도 향상)
        for inputs, labels in val_loader:
            outputs = model_lstm_random(inputs)  # 모델에 입력 데이터를 넣어 예측값 계산
            loss = loss_fn(outputs.squeeze(), labels)  # 손실 계산

            val_loss += loss.item()  # 현재 배치 손실을 누적
            predicted = (outputs.squeeze() > 0.5).float()  # 예측값 기반으로 정답 판단
            val_correct += (predicted == labels).sum().item()  # 맞춘 개수 누적
            val_total += labels.size(0)  # 전체 데이터 수 누적

    val_losses.append(val_loss / len(val_loader))  # 에포크별 평균 검증 손실 기록
    val_accs.append(val_correct / val_total)  # 에포크별 검증 정확도 기록

    # 손실 개선 여부 확인
    if val_loss < best_loss - min_delta:
        best_loss = val_loss
        patience_counter = 0
        # 최적의 모델 가중치 저장
        torch.save(model_lstm_random.state_dict(), 'model_lstm_random.pth')
        print(f"Validation loss improved. Saving model weights...")
    else:
        patience_counter += 1
        print(f"Validation loss did not improve. Patience counter: {patience_counter}/{patience}")

    # 조기 종료 조건 확인
    if patience_counter >= patience:
        print(f"Early stopping triggered after {epoch+1} epochs.")
        break

    
    # 에포크별 결과 출력
    print(f"Epoch {epoch+1}/{epochs} - "
          f"T_Loss: {train_losses[-1]:.4f}, T_Acc: {train_accs[-1]:.4f} - "
          f"V_Loss: {val_losses[-1]:.4f}, V_Acc: {val_accs[-1]:.4f}")

#-------------------------------------------------------------------------------

In [ ]:
# gensim 형식으로 저장할 파일 경로
word2vec_file_path = 'model_lstm_random.txt'

# 모델의 임베딩 가중치와 단어-인덱스 매핑 준비
# 1. 모델 변수명을 'model_lstm_cnn_word2vec'로 수정
vectors = model_lstm_random.embedding.weight.detach().cpu().numpy()

# 2. 모델 훈련 시 사용했던 실제 'word_to_index' 딕셔너리를 활용하여
#    'index_to_word' 딕셔너리를 생성합니다.
#    (이 부분은 실제 훈련 코드에 있는 'word_to_index' 변수명으로 교체해야 합니다.)
#    예시: word_to_index = {'<pad>': 0, '<unk>': 1, '영화': 2, ...}
#    - word_to_index 변수가 없으면 이 코드는 에러를 발생시킵니다.
index_to_word = {index: word for word, index in word_to_index.items()}


# 파일에 쓰기
with open(word2vec_file_path, 'w') as f:
    # 1. 파일의 첫 줄에 '전체 단어 수', '벡터 차원'을 기록합니다.
    #    gensim이 이 정보를 보고 파일 형식을 판단합니다.
    vocab_size, word_vector_dim = vectors.shape
    f.write(f'{vocab_size} {word_vector_dim}\n')
    
    # 2. 각 단어의 벡터 값을 공백으로 구분하여 한 줄씩 기록합니다.
    for i in range(vocab_size):
        # 3. index_to_word.get(i, '<unk>')를 사용하여 없는 인덱스도 안전하게 처리
        word = index_to_word.get(i, '<unk>')
        vector_str = ' '.join(map(str, vectors[i, :]))
        f.write(f'{word} {vector_str}\n')

print(f"'{word2vec_file_path}' 파일에 임베딩 가중치가 저장되었습니다.")

In [ ]:
test_dataset = TensorDataset(X_test_tensor, torch.tensor(y_test, dtype=torch.float32))
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

model_lstm_random.eval()
test_loss = 0.0
test_correct = 0
test_total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model_lstm_random(inputs)
        loss = loss_fn(outputs.squeeze(), labels)

        test_loss += loss.item()
        predicted = (outputs.squeeze() > 0.5).float()
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)

print(f"Test Loss: {test_loss / len(test_loader):.4f}, Test Accuracy: {test_correct / test_total:.4f}")

In [ ]:
import matplotlib.pyplot as plt

epochs_range = range(1, len(train_losses) + 1)

# "bo"는 "파란색 점"입니다
plt.plot(epochs_range, train_losses, 'bo', label='Training loss')
# b는 "파란 실선"입니다
plt.plot(epochs_range, val_losses, 'b', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf()   # 그림을 초기화합니다

plt.plot(epochs_range, train_accs, 'bo', label='Training accuracy')
plt.plot(epochs_range, val_accs, 'b', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
#8) 한국어 Word2vec 임베딩 활용하여 성능 개선

한국어 Word2Vec은 /data 폴더 안에 있는 word2vec_ko.model을 활용하세요.
한국어 Word2Vec을 활용할 때는 load_word2vec_format() 형태가 아닌 load() 형태로 모델을 불러와주세요. 또한 모델을 활용할 때에는 아래 예시와 같이 .wv를 붙여서 활용합니다. 좀더 자세한 활용법에 대해선 다음 링크들을 참조해주세요. 참고 링크1, 참고 링크2
# 예시 코드
from gensim.models.keyedvectors import Word2VecKeyedVectors
word_vectors = Word2VecKeyedVectors.load(word2vec_file_path)
vector = word_vectors.wv[‘끝’]

In [ ]:
!pip install gensim

In [ ]:
# !mkdir -p sentiment_classification/data
# !pip install gensim==4.3.2
# !pip install scipy==1.12.0 numpy==1.26.3

In [ ]:
# embedding_layer = model.embedding
# weights = embedding_layer.weight.detach().cpu().numpy()

# print(weights.shape)  # shape: (vocab_size, embedding_dim)


#!mkdir ~/work/sentiment_classification/word2vec_emb

In [ ]:
import os
from gensim.models import Word2Vec

# 경로를 환경 변수와 결합하여 전체 경로를 만듭니다.
word2vec_file_path = os.path.join(os.getenv('HOME'), 'work/sentiment_classification/data/word2vec_ko.model')

# load() 함수를 사용하여 한국어 Word2Vec 모델 불러오기
try:
    word2vec_ko = Word2Vec.load(word2vec_file_path)
    print("한국어 Word2Vec 모델을 성공적으로 로드했습니다.")

    # .wv를 붙여서 활용
    vector = word2vec_ko.wv['사랑']
    print(f"'사랑' 단어의 벡터 차원: {vector.shape}")
    print(f"'사랑'과 유사한 단어: {word2vec_ko.wv.similar_by_word('사랑')}")

except FileNotFoundError:
    print(f"오류: {word2vec_file_path} 경로에 파일이 존재하지 않습니다.")
except Exception as e:
    print(f"모델 로드 중 오류가 발생했습니다: {e}")
# 전체 워드 벡터의 차원 확인
print(f"전체 워드 벡터의 shape: {word2vec_ko.wv.vectors.shape}")

In [ ]:
# Word2Vec 모델의 워드 벡터를 embedding_matrix에 복사
vocab_size = 10000  # 모델이 사용할 총 단어의 개수
#word_vector_dim = 100  # 워드 벡터의 차원 

# Word2Vec 모델의 벡터 차원 확인
word_vector_dim = word2vec_ko.wv.vector_size  # Word2Vec 모델의 실제 차원을 사용
print(f"Word2Vec 모델의 워드 벡터 차원: {word_vector_dim}")

#embedding_matrix = np.random.rand(vocab_size, word_vector_dim)  # 임의의 값으로 초기화된 임베딩 행렬 생성
embedding_matrix = np.zeros((vocab_size, word_vector_dim))


# # embedding_matrix에 Word2Vec 워드 벡터를 단어 하나씩마다 차례차례 카피한다.
# for i in range(4, vocab_size):  # 단어 인덱스 4부터 시작하여 모든 단어에 대해 반복
#     if index_to_word[i] in word2vec_ko.wv:  # 현재 단어가 Word2Vec 모델에 포함되어 있는지 확인
#         embedding_matrix[i] = word2vec_ko.wv[index_to_word[i]]  
#         # 포함되어 있다면, 해당 단어의 워드 벡터를 embedding_matrix의 i번째 행에 복사




        # embedding_matrix에 Word2Vec 워드 벡터를 복사
for i in range(4, vocab_size):
    word = index_to_word[i]
    if word in word2vec_ko.wv:
        embedding_matrix[i] = word2vec_ko.wv[word]

In [ ]:
# 1_2 LSTM 기반 감성 분석 모델 정의 및 구조 출력
import torch  # 딥러닝 프레임워크 PyTorch 라이브러리
import torch.nn as nn  # 신경망 모듈
import torch.nn.functional as F  # 신경망 함수 모음

vocab_size = 10000  # 모델이 다룰 수 있는 단어의 총 개수
word_vector_dim = 100  # 각 단어를 표현하는 워드 벡터의 차원

# 모델 설계: nn.Module을 상속받아 SentimentModel 클래스 정의
class SentimentModel(nn.Module):
    # 모델의 레이어(Layer)들을 정의
    def __init__(self, vocab_size, word_vector_dim):
        super(SentimentModel, self).__init__()  # nn.Module의 생성자 호출
        self.embedding = nn.Embedding(vocab_size, word_vector_dim)  # 단어를 워드 벡터로 변환하는 임베딩 레이어
        self.lstm = nn.LSTM(word_vector_dim, 8, batch_first=True)  # 워드 벡터 시퀀스를 처리하는 LSTM 레이어
        self.fc1 = nn.Linear(8, 8)  # LSTM의 최종 출력을 받아서 처리하는 첫 번째 완전 연결 레이어
        self.fc2 = nn.Linear(8, 1)  # 최종 출력을 0과 1 사이의 값으로 만드는 두 번째 완전 연결 레이어

    # 모델의 순전파(forward) 로직을 정의
    def forward(self, x):
        x = self.embedding(x)  # 입력(단어 인덱스)을 임베딩 레이어에 통과시켜 워드 벡터로 변환
        x, (hn, cn) = self.lstm(x)  # 임베딩 벡터 시퀀스를 LSTM에 통과
        x = x[:, -1, :]  # LSTM의 최종 시점(sequence의 마지막)의 은닉 상태를 가져옴
        x = F.relu(self.fc1(x))  # 첫 번째 완전 연결 레이어에 통과시키고 ReLU 활성화 함수 적용
        x = torch.sigmoid(self.fc2(x))  # 두 번째 완전 연결 레이어에 통과시키고 시그모이드 활성화 함수 적용 (0~1 사이 값)
        return x  # 최종 출력값 반환

model_lstm_word2vec = SentimentModel(vocab_size, word_vector_dim)  # 정의된 클래스를 바탕으로 모델 객체 생성
print(model_lstm_word2vec)  # 모델의 레이어 구조와 파라미터 정보 출력

In [ ]:
# PyTorch를 이용한 감성 분석 모델 훈련
import torch.optim as optim  # 최적화 도구 모음
import torch.nn.functional as F  # 활성화 함수, 손실 함수 등
from torch.utils.data import DataLoader, TensorDataset  # 데이터 로딩 유틸리티

#모델 임베딩 초기화로 Word2vec 임베딩 매트릭스 적용
model_lstm_word2vec.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))


optimizer = optim.Adam(model_lstm_word2vec.parameters())  # 모델의 파라미터를 업데이트할 Adam 최적화 도구 설정
loss_fn = torch.nn.BCELoss()  # 이진 분류에 적합한 Binary Cross Entropy 손실 함수 설정

partial_X_train_tensor = partial_X_train.detach().clone().to(torch.long)  # 훈련용 입력 텐서를 복사하고 데이터 타입을 long으로 변환
partial_y_train_tensor = torch.tensor(partial_y_train, dtype=torch.float)  # 훈련용 라벨 데이터를 PyTorch 텐서로 변환 (BCELoss를 위해 float 타입 사용)

X_val_tensor = X_val.detach().clone().to(torch.long)  # 검증용 입력 텐서를 복사하고 데이터 타입을 long으로 변환
y_val_tensor = torch.tensor(y_val, dtype=torch.float)  # 검증용 라벨 데이터를 PyTorch 텐서로 변환

train_dataset = TensorDataset(partial_X_train_tensor, partial_y_train_tensor)  # 훈련 데이터와 라벨을 묶어 데이터셋 생성
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)  # 검증 데이터와 라벨을 묶어 데이터셋 생성

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)  # 훈련용 데이터로더 생성 (배치 크기 512, 데이터 섞기)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)  # 검증용 데이터로더 생성 (배치 크기 512, 데이터 섞지 않음)

epochs = 30  # 훈련 반복 횟수 설정
train_losses = []  # 에포크별 훈련 손실을 저장할 리스트
val_losses = []  # 에포크별 검증 손실을 저장할 리스트
train_accs = []  # 에포크별 훈련 정확도를 저장할 리스트
val_accs = []  # 에포크별 검증 정확도를 저장할 리스트

# 조기 종료 관련 하이퍼파라미터
patience = 5          # 검증 손실이 5번 연속으로 개선되지 않으면 훈련 중단
min_delta = 0.0001    # 최소한 이 값 이상 손실이 감소해야 개선으로 인정

best_loss = np.Inf    # 초기 최저 손실값 설정
patience_counter = 0  # 손실 개선 실패 횟수 카운터

for epoch in range(epochs):
    # 훈련 단계 시작
    model_lstm_word2vec.train()  # 모델을 훈련 모드로 설정
    running_loss = 0.0  # 현재 에포크의 누적 손실 초기화
    correct = 0  # 현재 에포크의 정답 수 초기화
    total = 0  # 현재 에포크의 전체 데이터 수 초기화

    for inputs, labels in train_loader:
        optimizer.zero_grad()  # 이전 스텝에서 계산된 기울기 초기화

        outputs = model_lstm_word2vec(inputs)  # 모델에 입력 데이터를 넣어 예측값 계산
        loss = loss_fn(outputs.squeeze(), labels)  # 예측값과 실제 라벨 간의 손실 계산
        loss.backward()  # 손실에 대한 기울기 역전파
        optimizer.step()  # 모델의 파라미터 업데이트

        running_loss += loss.item()  # 현재 배치 손실을 누적
        predicted = (outputs.squeeze() > 0.5).float()  # 예측값이 0.5보다 크면 긍정(1), 아니면 부정(0)으로 판단
        correct += (predicted == labels).sum().item()  # 맞춘 개수 누적
        total += labels.size(0)  # 전체 데이터 수 누적

    train_losses.append(running_loss / len(train_loader))  # 에포크별 평균 훈련 손실 기록
    train_accs.append(correct / total)  # 에포크별 훈련 정확도 기록

    # 검증 단계 시작
    model_lstm_word2vec.eval()  # 모델을 평가 모드로 설정
    val_loss = 0.0  # 현재 에포크의 누적 검증 손실 초기화
    val_correct = 0  # 현재 에포크의 정답 수 초기화
    val_total = 0  # 현재 에포크의 전체 데이터 수 초기화

    with torch.no_grad():  # 기울기 계산을 비활성화 (메모리 절약, 연산 속도 향상)
        for inputs, labels in val_loader:
            outputs = model_lstm_word2vec(inputs)  # 모델에 입력 데이터를 넣어 예측값 계산
            loss = loss_fn(outputs.squeeze(), labels)  # 손실 계산

            val_loss += loss.item()  # 현재 배치 손실을 누적
            predicted = (outputs.squeeze() > 0.5).float()  # 예측값 기반으로 정답 판단
            val_correct += (predicted == labels).sum().item()  # 맞춘 개수 누적
            val_total += labels.size(0)  # 전체 데이터 수 누적

    val_losses.append(val_loss / len(val_loader))  # 에포크별 평균 검증 손실 기록
    val_accs.append(val_correct / val_total)  # 에포크별 검증 정확도 기록

    val_loss = val_losses[-1] # 현재 에포크의 검증 손실

    # 손실 개선 여부 확인
    if val_loss < best_loss - min_delta:
        best_loss = val_loss
        patience_counter = 0
        # 최적의 모델 가중치 저장
        torch.save(model_lstm_word2vec.state_dict(), 'model_lstm_word2vec.pth')
        print(f"Validation loss improved. Saving model weights...")
    else:
        patience_counter += 1
        print(f"Validation loss did not improve. Patience counter: {patience_counter}/{patience}")

    # 조기 종료 조건 확인
    if patience_counter >= patience:
        print(f"Early stopping triggered after {epoch+1} epochs.")
        break
    
    # 에포크별 결과 출력
    print(f"Epoch {epoch+1}/{epochs} - "
          f"T_Loss: {train_losses[-1]:.4f}, T_Acc: {train_accs[-1]:.4f} - "
          f"V_Loss: {val_losses[-1]:.4f}, V_Acc: {val_accs[-1]:.4f}")

In [ ]:
# gensim 형식으로 저장할 파일 경로
word2vec_file_path = 'model_lstm_word2vec.txt'

# 모델의 임베딩 가중치와 단어-인덱스 매핑 준비
# 1. 모델 변수명을 'model_lstm_cnn_word2vec'로 수정
vectors = model_lstm_word2vec.embedding.weight.detach().cpu().numpy()

# 2. 모델 훈련 시 사용했던 실제 'word_to_index' 딕셔너리를 활용하여
#    'index_to_word' 딕셔너리를 생성합니다.
#    (이 부분은 실제 훈련 코드에 있는 'word_to_index' 변수명으로 교체해야 합니다.)
#    예시: word_to_index = {'<pad>': 0, '<unk>': 1, '영화': 2, ...}
#    - word_to_index 변수가 없으면 이 코드는 에러를 발생시킵니다.
index_to_word = {index: word for word, index in word_to_index.items()}


# 파일에 쓰기
with open(word2vec_file_path, 'w') as f:
    # 1. 파일의 첫 줄에 '전체 단어 수', '벡터 차원'을 기록합니다.
    #    gensim이 이 정보를 보고 파일 형식을 판단합니다.
    vocab_size, word_vector_dim = vectors.shape
    f.write(f'{vocab_size} {word_vector_dim}\n')
    
    # 2. 각 단어의 벡터 값을 공백으로 구분하여 한 줄씩 기록합니다.
    for i in range(vocab_size):
        # 3. index_to_word.get(i, '<unk>')를 사용하여 없는 인덱스도 안전하게 처리
        word = index_to_word.get(i, '<unk>')
        vector_str = ' '.join(map(str, vectors[i, :]))
        f.write(f'{word} {vector_str}\n')

print(f"'{word2vec_file_path}' 파일에 임베딩 가중치가 저장되었습니다.")

In [ ]:
test_dataset = TensorDataset(X_test_tensor, torch.tensor(y_test, dtype=torch.float32))
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

model_lstm_word2vec.eval()
test_loss = 0.0
test_correct = 0
test_total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model_lstm_word2vec(inputs)
        loss = loss_fn(outputs.squeeze(), labels)

        test_loss += loss.item()
        predicted = (outputs.squeeze() > 0.5).float()
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)

print(f"Test Loss: {test_loss / len(test_loader):.4f}, Test Accuracy: {test_correct / test_total:.4f}")

In [ ]:
import matplotlib.pyplot as plt

epochs_range = range(1, len(train_losses) + 1)

# "bo"는 "파란색 점"입니다
plt.plot(epochs_range, train_losses, 'bo', label='Training loss')
# b는 "파란 실선"입니다
plt.plot(epochs_range, val_losses, 'b', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf()   # 그림을 초기화합니다

plt.plot(epochs_range, train_accs, 'bo', label='Training accuracy')
plt.plot(epochs_range, val_accs, 'b', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
#CNN 모델 학습

In [ ]:
# 2_1 1D CNN을 이용한 감성 분석 모델 정의
import torch  # 딥러닝 프레임워크 PyTorch 라이브러리
from torch import nn  # 신경망 모듈
import torch.nn.functional as F  # 신경망 함수 모음 (시그모이드 함수를 위해 필요)

# SentenceClassifier_Cnn 클래스 정의 (nn.Module 상속)
class SentenceClassifier_Cnn(nn.Module):
    # 모델의 레이어(Layer)들을 정의하는 생성자
    def __init__(self, embedding_matrix, filter_sizes, max_length, dropout=0.5):
        super().__init__()  # nn.Module의 생성자 호출

        # 임베딩 레이어
        self.embedding = nn.Embedding(
            num_embeddings=embedding_matrix.shape[0],
            embedding_dim=embedding_matrix.shape[1]
        )
        # 외부에서 전달받은 행렬의 값을 임베딩 레이어의 가중치로 복사
        self.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        # 임베딩 레이어의 차원(word_vector_dim) 추출
        embedding_dim = self.embedding.weight.shape[1]
        num_filters = 128  # out_channels 수를 128로 설정

        conv = []
        for size in filter_sizes:  # filter_sizes (예: [3, 4, 5])에 따라 여러 개의 CNN 필터를 만듦
            conv.append(
                nn.Sequential(  # 여러 레이어를 묶어서 하나의 모듈로 구성
                    nn.Conv1d(  # 1차원 컨볼루션 레이어
                        in_channels=embedding_dim,  # 입력 채널 수: 워드 벡터의 차원
                        out_channels=num_filters,  # 출력 채널 수: 각 필터에서 하나의 특징을 추출
                        kernel_size=size  # 필터의 크기 (ex: 3, 4, 5)
                    ),
                    nn.ReLU(),  # 비선형성을 추가하는 활성화 함수
                    nn.AdaptiveMaxPool1d(1),  # 최대 풀링으로 가장 중요한 특징 추출
                )
            )
        self.conv_filters = nn.ModuleList(conv)  # 여러 컨볼루션 모듈을 리스트 형태로 저장

        num_filters = 128
        output_size = num_filters * len(filter_sizes) # 128 * 3 = 384 # 필터의 개수가 최종 출력의 크기가 됨
        self.pre_classifier = nn.Linear(output_size, 64)  # 첫 번째 완전 연결 레이어
        self.dropout = nn.Dropout(dropout)  # 과적합을 방지하는 드롭아웃 레이어
        self.classifier = nn.Linear(64, 1)  # 최종 분류를 위한 완전 연결 레이어 (출력 1개)

    # 모델의 순전파(forward) 로직을 정의
    def forward(self, inputs):
        embeddings = self.embedding(inputs)  # 입력(단어 인덱스)을 임베딩 레이어에 통과
        embeddings = embeddings.permute(0, 2, 1)  # Conv1d 입력에 맞게 차원 변경 (배치, 차원, 길이)

        conv_outputs = [conv(embeddings) for conv in self.conv_filters]  # 각 필터를 통과한 결과물을 리스트에 저장
        concat_outputs = torch.cat([conv.squeeze(-1) for conv in conv_outputs], dim=1)  # 필터 결과물을 하나로 합침

        logits = self.pre_classifier(concat_outputs)  # 첫 번째 완전 연결 레이어 통과
        logits = self.dropout(logits)  # 드롭아웃 적용
        logits = self.classifier(logits)  # 최종 분류 레이어 통과
        
        return torch.sigmoid(logits)  # 이진 분류를 위해 시그모이드 활성화 함수 적용

In [ ]:
# 델 인스턴스화를 위한 변수 설정
# 이전 LSTM 실험에서 사용한 값들을 기준으로 설정
vocab_size = 10000 
word_vector_dim = 16
max_length = 41  # 예시 값 (실제 데이터의 max_length로 대체 필요)

# 실험을 위한 랜덤 임베딩 행렬 생성
# Word2Vec을 사용하지 않고 랜덤 값으로 초기화
random_embedding_matrix = np.random.rand(vocab_size, word_vector_dim)

# 1D CNN 모델 인스턴스화 (랜덤 임베딩 사용)
model_cnn_random = SentenceClassifier_Cnn(
    embedding_matrix=random_embedding_matrix, 
    filter_sizes=[2, 3, 4], # 예시: 다양한 크기의 필터 사용
    max_length=max_length,
    dropout=0.5
)

# 6. 모델 구조 출력
print("첫 번째 모델 (랜덤 임베딩) 구조:")
print(model_cnn_random)

In [ ]:
#2_1 1D CNN을 이용한 감성 분석 모델 훈련
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# --- 아래 코드는 제거해야 합니다. cnn 1 실험은 Word2Vec을 사용하지 않습니다. ---
# model.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
# ---------------------------------------------------------------------

# model 변수에는 'model_cnn_random' 객체가 할당되어 있다고 가정
# optimizer를 정의할 때 이 모델을 사용해야 합니다.
optimizer = optim.Adam(model_cnn_random.parameters())
loss_fn = torch.nn.BCELoss()

partial_X_train_tensor = partial_X_train.detach().clone().to(torch.long)
partial_y_train_tensor = torch.tensor(partial_y_train, dtype=torch.float)

X_val_tensor = X_val.detach().clone().to(torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.float)

train_dataset = TensorDataset(partial_X_train_tensor, partial_y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)

epochs = 30
train_losses = []
val_losses = []
train_accs = []
val_accs = []

# 조기 종료 관련 하이퍼파라미터
patience = 5          # 검증 손실이 5번 연속으로 개선되지 않으면 훈련 중단
min_delta = 0.0001    # 최소한 이 값 이상 손실이 감소해야 개선으로 인정
best_loss = np.Inf    # 초기 최저 손실값 설정
patience_counter = 0  # 손실 개선 실패 횟수 카운터

for epoch in range(epochs):
    # 훈련 단계 시작
    model_cnn_random.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model_cnn_random(inputs)
        loss = loss_fn(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        predicted = (outputs.squeeze() > 0.5).float()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_losses.append(running_loss / len(train_loader))
    train_accs.append(correct / total)

    # 검증 단계 시작
    model_cnn_random.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model_cnn_random(inputs)
            loss = loss_fn(outputs.squeeze(), labels)
            val_loss += loss.item()
            predicted = (outputs.squeeze() > 0.5).float()
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_losses.append(val_loss / len(val_loader))
    val_accs.append(val_correct / val_total)

    val_loss = val_losses[-1] # 현재 에포크의 검증 손실

    # 손실 개선 여부 확인
    if val_loss < best_loss - min_delta:
        best_loss = val_loss
        patience_counter = 0
        # 최적의 모델 가중치 저장
        torch.save(model_cnn_random.state_dict(), 'model_cnn_random.pth')
        print(f"Validation loss improved. Saving model weights...")
    else:
        patience_counter += 1
        print(f"Validation loss did not improve. Patience counter: {patience_counter}/{patience}")

    # 조기 종료 조건 확인
    if patience_counter >= patience:
        print(f"Early stopping triggered after {epoch+1} epochs.")
        break


        
    print(f"Epoch {epoch+1}/{epochs} - "
          f"T_Loss: {train_losses[-1]:.4f}, T_Acc: {train_accs[-1]:.4f} - "
          f"V_Loss: {val_losses[-1]:.4f}, V_Acc: {val_accs[-1]:.4f}")


In [ ]:
# gensim 형식으로 저장할 파일 경로
word2vec_file_path = 'model_cnn_random.txt'

# 모델의 임베딩 가중치와 단어-인덱스 매핑 준비
# 1. 모델 변수명을 'model_lstm_cnn_word2vec'로 수정
vectors = model_cnn_random.embedding.weight.detach().cpu().numpy()

# 2. 모델 훈련 시 사용했던 실제 'word_to_index' 딕셔너리를 활용하여
#    'index_to_word' 딕셔너리를 생성합니다.
#    (이 부분은 실제 훈련 코드에 있는 'word_to_index' 변수명으로 교체해야 합니다.)
#    예시: word_to_index = {'<pad>': 0, '<unk>': 1, '영화': 2, ...}
#    - word_to_index 변수가 없으면 이 코드는 에러를 발생시킵니다.
index_to_word = {index: word for word, index in word_to_index.items()}


# 파일에 쓰기
with open(word2vec_file_path, 'w') as f:
    # 1. 파일의 첫 줄에 '전체 단어 수', '벡터 차원'을 기록합니다.
    #    gensim이 이 정보를 보고 파일 형식을 판단합니다.
    vocab_size, word_vector_dim = vectors.shape
    f.write(f'{vocab_size} {word_vector_dim}\n')
    
    # 2. 각 단어의 벡터 값을 공백으로 구분하여 한 줄씩 기록합니다.
    for i in range(vocab_size):
        # 3. index_to_word.get(i, '<unk>')를 사용하여 없는 인덱스도 안전하게 처리
        word = index_to_word.get(i, '<unk>')
        vector_str = ' '.join(map(str, vectors[i, :]))
        f.write(f'{word} {vector_str}\n')

print(f"'{word2vec_file_path}' 파일에 임베딩 가중치가 저장되었습니다.")

In [ ]:
test_dataset = TensorDataset(X_test_tensor, torch.tensor(y_test, dtype=torch.float32))
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

model_cnn_random.eval()
test_loss = 0.0
test_correct = 0
test_total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model_cnn_random(inputs)
        loss = loss_fn(outputs.squeeze(), labels)

        test_loss += loss.item()
        predicted = (outputs.squeeze() > 0.5).float()
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)

print(f"Test Loss: {test_loss / len(test_loader):.4f}, Test Accuracy: {test_correct / test_total:.4f}")

In [ ]:
import matplotlib.pyplot as plt

epochs_range = range(1, len(train_losses) + 1)

# "bo"는 "파란색 점"입니다
plt.plot(epochs_range, train_losses, 'bo', label='Training loss')
# b는 "파란 실선"입니다
plt.plot(epochs_range, val_losses, 'b', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf()   # 그림을 초기화합니다

plt.plot(epochs_range, train_accs, 'bo', label='Training accuracy')
plt.plot(epochs_range, val_accs, 'b', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# 2_2 1D CNN 모델 인스턴스화를 위한 변수 설정(word2vec 임베딩 사용)
# Word2Vec 모델의 차원에 맞춰 word_vector_dim을 100으로 설정
vocab_size = 10000 
word_vector_dim = 100
max_length = 41  # pad_sequences의 maxlen 값 사용

# 1. Word2Vec 모델 로드 및 임베딩 행렬 생성
word2vec_file_path = os.path.join(os.getenv('HOME'), 'work/sentiment_classification/data/word2vec_ko.model')
word2vec_ko = Word2Vec.load(word2vec_file_path)

# word_index = ...  # 단어 인덱스 딕셔너리
# index_to_word = {index: word for word, index in word_index.items()}

# Word2Vec으로 채워진 임베딩 행렬 생성
embedding_matrix = np.random.rand(vocab_size, word_vector_dim)
for i in range(4, vocab_size):
    if index_to_word[i] in word2vec_ko.wv:
        embedding_matrix[i] = word2vec_ko.wv[index_to_word[i]]

# 2. 1D CNN 모델 인스턴스화 (Word2Vec 임베딩 사용)
# SentenceClassifier_Cnn 클래스는 이미 정의되어 있다고 가정
model_cnn_word2vec = SentenceClassifier_Cnn(
    embedding_matrix=embedding_matrix, 
    filter_sizes=[2, 3, 4],
    max_length=max_length,
    dropout=0.5
)

# 모델 구조 출력 (확인용)
print("--- cnn 2: Word2Vec 임베딩 행렬 사용 ---")
print(model_cnn_word2vec)

In [ ]:
#2_2 1D CNN을 이용한 감성 분석 모델 훈련2 word2vec
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# 이 코드는 Word2Vec 임베딩 행렬로 모델을 초기화하는 부분입니다.
# cnn 2 실험에 필수적이므로 그대로 유지해야 합니다.
# model_cnn_word2vec.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))

# model 변수에는 'model_cnn_word2ve' 객체가 할당되어 있다고 가정
# optimizer를 정의할 때 이 모델을 사용해야 합니다.
# --- 이 부분을 수정해야 합니다. ---
optimizer = optim.Adam(model_cnn_word2vec.parameters())
loss_fn = torch.nn.BCELoss()

partial_X_train_tensor = partial_X_train.detach().clone().to(torch.long)
partial_y_train_tensor = torch.tensor(partial_y_train, dtype=torch.float)

X_val_tensor = X_val.detach().clone().to(torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.float)

train_dataset = TensorDataset(partial_X_train_tensor, partial_y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)

epochs = 30
train_losses = []
val_losses = []
train_accs = []
val_accs = []

# 조기 종료 관련 하이퍼파라미터
patience = 5          # 검증 손실이 5번 연속으로 개선되지 않으면 훈련 중단
min_delta = 0.0001    # 최소한 이 값 이상 손실이 감소해야 개선으로 인정
best_loss = np.Inf    # 초기 최저 손실값 설정
patience_counter = 0  # 손실 개선 실패 횟수 카운터

for epoch in range(epochs):
    # 훈련 단계 시작
    # --- 이 부분도 수정해야 합니다. ---
    model_cnn_word2vec.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        # --- 이 부분도 수정해야 합니다. ---
        outputs = model_cnn_word2vec(inputs)
        loss = loss_fn(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        predicted = (outputs.squeeze() > 0.5).float()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_losses.append(running_loss / len(train_loader))
    train_accs.append(correct / total)

    # 검증 단계 시작
    model_cnn_word2vec.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model_cnn_word2vec(inputs)
            loss = loss_fn(outputs.squeeze(), labels)
            val_loss += loss.item()
            predicted = (outputs.squeeze() > 0.5).float()
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_losses.append(val_loss / len(val_loader))
    val_accs.append(val_correct / val_total)

    val_loss = val_losses[-1] # 현재 에포크의 검증 손실

    # 손실 개선 여부 확인
    if val_loss < best_loss - min_delta:
        best_loss = val_loss
        patience_counter = 0
        # 최적의 모델 가중치 저장
        torch.save(model_cnn_word2vec.state_dict(), 'model_cnn_word2vec.pth')
        print(f"Validation loss improved. Saving model weights...")
    else:
        patience_counter += 1
        print(f"Validation loss did not improve. Patience counter: {patience_counter}/{patience}")

    # 조기 종료 조건 확인
    if patience_counter >= patience:
        print(f"Early stopping triggered after {epoch+1} epochs.")
        break

    
    print(f"Epoch {epoch+1}/{epochs} - "
          f"T_Loss: {train_losses[-1]:.4f}, T_Acc: {train_accs[-1]:.4f} - "
          f"V_Loss: {val_losses[-1]:.4f}, V_Acc: {val_accs[-1]:.4f}")

In [ ]:
# gensim 형식으로 저장할 파일 경로
word2vec_file_path = 'model_cnn_word2vec.txt'

# 모델의 임베딩 가중치와 단어-인덱스 매핑 준비
# 1. 모델 변수명을 'model_lstm_cnn_word2vec'로 수정
vectors = model_cnn_word2vec.embedding.weight.detach().cpu().numpy()

# 2. 모델 훈련 시 사용했던 실제 'word_to_index' 딕셔너리를 활용하여
#    'index_to_word' 딕셔너리를 생성합니다.
#    (이 부분은 실제 훈련 코드에 있는 'word_to_index' 변수명으로 교체해야 합니다.)
#    예시: word_to_index = {'<pad>': 0, '<unk>': 1, '영화': 2, ...}
#    - word_to_index 변수가 없으면 이 코드는 에러를 발생시킵니다.
index_to_word = {index: word for word, index in word_to_index.items()}


# 파일에 쓰기
with open(word2vec_file_path, 'w') as f:
    # 1. 파일의 첫 줄에 '전체 단어 수', '벡터 차원'을 기록합니다.
    #    gensim이 이 정보를 보고 파일 형식을 판단합니다.
    vocab_size, word_vector_dim = vectors.shape
    f.write(f'{vocab_size} {word_vector_dim}\n')
    
    # 2. 각 단어의 벡터 값을 공백으로 구분하여 한 줄씩 기록합니다.
    for i in range(vocab_size):
        # 3. index_to_word.get(i, '<unk>')를 사용하여 없는 인덱스도 안전하게 처리
        word = index_to_word.get(i, '<unk>')
        vector_str = ' '.join(map(str, vectors[i, :]))
        f.write(f'{word} {vector_str}\n')

print(f"'{word2vec_file_path}' 파일에 임베딩 가중치가 저장되었습니다.")

In [ ]:
test_dataset = TensorDataset(X_test_tensor, torch.tensor(y_test, dtype=torch.float32))
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

model_cnn_word2vec.eval()
test_loss = 0.0
test_correct = 0
test_total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model_cnn_word2vec(inputs)
        loss = loss_fn(outputs.squeeze(), labels)

        test_loss += loss.item()
        predicted = (outputs.squeeze() > 0.5).float()
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)

print(f"Test Loss: {test_loss / len(test_loader):.4f}, Test Accuracy: {test_correct / test_total:.4f}")

In [ ]:
import matplotlib.pyplot as plt

epochs_range = range(1, len(train_losses) + 1)

# "bo"는 "파란색 점"입니다
plt.plot(epochs_range, train_losses, 'bo', label='Training loss')
# b는 "파란 실선"입니다
plt.plot(epochs_range, val_losses, 'b', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf()   # 그림을 초기화합니다

plt.plot(epochs_range, train_accs, 'bo', label='Training accuracy')
plt.plot(epochs_range, val_accs, 'b', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
3-1

In [ ]:
3-1
import torch
from torch import nn
import torch.nn.functional as F

class SentenceClassifier_LstmCnn(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, num_layers, dropout, bidirectional, filter_sizes, max_length):
        super().__init__()

        # 1. 임베딩 레이어: 단어를 벡터로 변환
        self.embedding = nn.Embedding(
            num_embeddings=embedding_matrix.shape[0],
            embedding_dim=embedding_matrix.shape[1]
        )
        
          # 외부에서 전달받은 행렬의 값을 임베딩 레이어의 가중치로 복사
        self.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        embedding_dim = self.embedding.weight.shape[1]
        # self.embedding.weight.requires_grad = True

        # 2. LSTM 레이어: 문장의 순차적인 흐름과 맥락을 학습
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            bidirectional=bidirectional,
            dropout=dropout,
            batch_first=True  # 입력 텐서의 첫 번째 차원이 배치 크기임을 명시
        )
        
        # LSTM의 출력 차원을 계산
        lstm_output_dim = hidden_dim * 2 if bidirectional else hidden_dim
        num_filters = 128  # out_channels 수를 128로 설정
        
        # 3. CNN 필터들: LSTM 출력에서 특징을 추출
        conv = []
        for size in filter_sizes:
            conv.append(
                nn.Sequential(
                    nn.Conv1d(
                        in_channels=lstm_output_dim,  # LSTM의 출력을 입력으로 사용
                        out_channels=num_filters,
                        kernel_size=size
                    ),
                    nn.ReLU(),
                    nn.AdaptiveMaxPool1d(1),
                )
            )
        self.conv_filters = nn.ModuleList(conv)
        
        # 4. 분류기 레이어: 추출된 특징으로 최종 분류
        output_size = num_filters * len(filter_sizes) # 128 * 3 = 384

        self.pre_classifier = nn.Linear(output_size, 64)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(64, 1)

    def forward(self, inputs):
        # (1) 임베딩 레이어 통과
        embeddings = self.embedding(inputs)  # [배치, 시퀀스 길이, 임베딩 차원]

        # (2) LSTM 레이어 통과
        lstm_output, (hidden, cell) = self.lstm(embeddings)
        # lstm_output: [배치, 시퀀스 길이, hidden_dim * 2 (양방향인 경우)]

        # (3) CNN 레이어 입력에 맞게 차원 변경
        # Conv1d를 위해 [배치, 채널 수, 시퀀스 길이] 형태로 변경
        lstm_output = lstm_output.permute(0, 2, 1)

        # (4) CNN 필터들 통과
        conv_outputs = [conv(lstm_output) for conv in self.conv_filters]
        concat_outputs = torch.cat([conv.squeeze(-1) for conv in conv_outputs], dim=1)

        # (5) 분류기 통과
        logits = self.pre_classifier(concat_outputs)
        logits = self.dropout(logits)
        logits = self.classifier(logits)

        # 최종 시그모이드 활성화 함수 적용
        return torch.sigmoid(logits)

In [ ]:
# 0. LSTM-CNN 모델 학습 코드의 목적
# 이 코드는 SentenceClassifier_LstmCnn 모델을 사용하여 영화 리뷰 감성 분석 태스크를 훈련하고 평가하는 과정을 수행한다.
# 훈련 손실(train_loss)과 정확도(train_accuracy), 그리고 검증 손실(validation_loss)과 정확도(validation_accuracy)를 매 에포크마다 계산하고 출력한다.

import torch
import torch.optim as optim  # 최적화 도구 모음
import torch.nn.functional as F  # 활성화 함수, 손실 함수 등
from torch.utils.data import DataLoader, TensorDataset  # 데이터 로딩 유틸리티
import numpy as np
# from SentenceClassifier_LstmCnn import SentenceClassifier_LstmCnn  # 모델 클래스 import
from tqdm import tqdm  # tqdm 라이브러리 import
import numpy as np

# 조기 종료 관련 하이퍼파라미터
patience = 5          # 검증 손실이 5번 연속으로 개선되지 않으면 훈련 중단
min_delta = 0.0001    # 최소한 이 값 이상 손실이 감소해야 개선으로 인정

best_loss = np.Inf    # 초기 최저 손실값 설정
patience_counter = 0  # 손실 개선 실패 횟수 카운터


# 모델 인스턴스화를 위한 변수 설정
# 이전 실험들과 일관성을 유지하거나 새로운 하이퍼파라미터를 설정한다.
vocab_size = 10000 
word_vector_dim = 16  # Word2Vec 임베딩을 위해 100으로 설정 (랜덤 실험 시에는 16)
max_length = 41
hidden_dim = 128  # LSTM의 은닉 상태 차원
num_layers = 2   # LSTM 레이어 수
dropout = 0.5  # 드롭아웃 확률
bidirectional = True # LSTM 양방향 설정
filter_sizes = [2, 3, 4] # CNN 필터 크기

# Word2Vec 임베딩 행렬을 사용한다고 가정하고, 이를 생성
# (실제 Word2Vec 모델 로드 및 임베딩 행렬 생성 코드는 이전에 수행되었어야 함)
# embedding_matrix = ...
embedding_matrix = np.random.rand(vocab_size, word_vector_dim)


# 모델 인스턴스화
model_lstm_cnn_random = SentenceClassifier_LstmCnn(
    embedding_matrix=embedding_matrix,  
    hidden_dim=hidden_dim,  
    num_layers=num_layers,  
    dropout=dropout,  
    bidirectional=bidirectional,  
    filter_sizes=filter_sizes,  
    max_length=max_length
)

# 1. 모델 임베딩 초기화 (Word2Vec 사용 시)
# 모델의 임베딩 레이어를 사전 학습된 Word2Vec 벡터로 덮어씌운다.
# 랜덤 임베딩을 사용하는 실험에서는 이 줄을 주석 처리해야 한다.
#model_lstm_cnn_random.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))

# 2. 옵티마이저와 손실 함수 설정
# Adam 최적화 도구와 이진 분류에 적합한 BCELoss 손실 함수를 사용한다.
optimizer = optim.Adam(model_lstm_cnn_random.parameters())
loss_fn = torch.nn.BCELoss()

# 3. 데이터 로더 준비
# 훈련 및 검증 데이터를 텐서로 변환하고 데이터셋, 데이터로더를 생성한다.
partial_X_train_tensor = partial_X_train.detach().clone().to(torch.long)
partial_y_train_tensor = torch.tensor(partial_y_train, dtype=torch.float)

X_val_tensor = X_val.detach().clone().to(torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.float)

train_dataset = TensorDataset(partial_X_train_tensor, partial_y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)

epochs = 30  # 훈련 반복 횟수 설정
train_losses = []  # 에포크별 훈련 손실을 저장할 리스트
val_losses = []    # 에포크별 검증 손실을 저장할 리스트
train_accs = []    # 에포크별 훈련 정확도를 저장할 리스트
val_accs = []      # 에포크별 검증 정확도를 저장할 리스트

# 4. 모델 훈련 및 검증 루프
for epoch in range(epochs):
    # 훈련 단계 시작
    model_lstm_cnn_random.train()  # 모델을 훈련 모드로 설정
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]"):
        optimizer.zero_grad()  # 이전 스텝에서 계산된 기울기 초기화
        outputs = model_lstm_cnn_random(inputs)  # 모델에 입력 데이터를 넣어 예측값 계산
        loss = loss_fn(outputs.squeeze(), labels)  # 예측값과 실제 라벨 간의 손실 계산
        loss.backward()  # 손실에 대한 기울기 역전파
        optimizer.step()  # 모델의 파라미터 업데이트

        running_loss += loss.item()
        predicted = (outputs.squeeze() > 0.5).float()  # 예측값이 0.5보다 크면 긍정(1), 아니면 부정(0)으로 판단
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_losses.append(running_loss / len(train_loader))
    train_accs.append(correct / total)

    # 검증 단계 시작
    model_lstm_cnn_random.eval()  # 모델을 평가 모드로 설정
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    

    with torch.no_grad():  # 기울기 계산 비활성화 (메모리 절약, 속도 향상)
        for inputs, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Validation]"):
            outputs = model_lstm_cnn_random(inputs)
            loss = loss_fn(outputs.squeeze(), labels)
            val_loss += loss.item()
            predicted = (outputs.squeeze() > 0.5).float()
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_losses.append(val_loss / len(val_loader))
    val_accs.append(val_correct / val_total)

    val_loss = val_losses[-1] # 현재 에포크의 검증 손실

    # 손실 개선 여부 확인
    if val_loss < best_loss - min_delta:
        best_loss = val_loss
        patience_counter = 0
        # 최적의 모델 가중치 저장
        torch.save(model_lstm_cnn_random.state_dict(), 'model_lstm_cnn_random.pth')
        print(f"Validation loss improved. Saving model weights...")
    else:
        patience_counter += 1
        print(f"Validation loss did not improve. Patience counter: {patience_counter}/{patience}")

    # 조기 종료 조건 확인
    if patience_counter >= patience:
        print(f"Early stopping triggered after {epoch+1} epochs.")
        break


    
    # 에포크별 결과 출력
    print(f"Epoch {epoch+1}/{epochs} - "
          f"T_Loss: {train_losses[-1]:.4f}, T_Acc: {train_accs[-1]:.4f} - "
          f"V_Loss: {val_losses[-1]:.4f}, V_Acc: {val_accs[-1]:.4f}")

In [ ]:
# gensim 형식으로 저장할 파일 경로
word2vec_file_path = 'model_lstm_cnn_random.txt'

# 모델의 임베딩 가중치와 단어-인덱스 매핑 준비
# 1. 모델 변수명을 'model_lstm_cnn_word2vec'로 수정
vectors = model_lstm_cnn_random.embedding.weight.detach().cpu().numpy()

# 2. 모델 훈련 시 사용했던 실제 'word_to_index' 딕셔너리를 활용하여
#    'index_to_word' 딕셔너리를 생성합니다.
#    (이 부분은 실제 훈련 코드에 있는 'word_to_index' 변수명으로 교체해야 합니다.)
#    예시: word_to_index = {'<pad>': 0, '<unk>': 1, '영화': 2, ...}
#    - word_to_index 변수가 없으면 이 코드는 에러를 발생시킵니다.
index_to_word = {index: word for word, index in word_to_index.items()}


# 파일에 쓰기
with open(word2vec_file_path, 'w') as f:
    # 1. 파일의 첫 줄에 '전체 단어 수', '벡터 차원'을 기록합니다.
    #    gensim이 이 정보를 보고 파일 형식을 판단합니다.
    vocab_size, word_vector_dim = vectors.shape
    f.write(f'{vocab_size} {word_vector_dim}\n')
    
    # 2. 각 단어의 벡터 값을 공백으로 구분하여 한 줄씩 기록합니다.
    for i in range(vocab_size):
        # 3. index_to_word.get(i, '<unk>')를 사용하여 없는 인덱스도 안전하게 처리
        word = index_to_word.get(i, '<unk>')
        vector_str = ' '.join(map(str, vectors[i, :]))
        f.write(f'{word} {vector_str}\n')

print(f"'{word2vec_file_path}' 파일에 임베딩 가중치가 저장되었습니다.")

In [ ]:
test_dataset = TensorDataset(X_test_tensor, torch.tensor(y_test, dtype=torch.float32))
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

model_lstm_cnn_random.load_state_dict(torch.load('model_lstm_cnn_random.pth'))
model_lstm_cnn_random.eval()
test_loss = 0.0
test_correct = 0
test_total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model_lstm_cnn_random(inputs)
        loss = loss_fn(outputs.squeeze(), labels)

        test_loss += loss.item()
        predicted = (outputs.squeeze() > 0.5).float()
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)

print(f"Test Loss: {test_loss / len(test_loader):.4f}, Test Accuracy: {test_correct / test_total:.4f}")

In [ ]:
import matplotlib.pyplot as plt

epochs_range = range(1, len(train_losses) + 1)

# "bo"는 "파란색 점"입니다
plt.plot(epochs_range, train_losses, 'bo', label='Training loss')
# b는 "파란 실선"입니다
plt.plot(epochs_range, val_losses, 'b', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf()   # 그림을 초기화합니다

plt.plot(epochs_range, train_accs, 'bo', label='Training accuracy')
plt.plot(epochs_range, val_accs, 'b', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
3-2

In [ ]:
# 3-2 0. LSTM-CNN 모델 학습 코드의 목적
# 이 코드는 SentenceClassifier_LstmCnn 모델을 사용하여 영화 리뷰 감성 분석 태스크를 훈련하고 평가하는 과정을 수행한다.
# 훈련 손실(train_loss)과 정확도(train_accuracy), 그리고 검증 손실(validation_loss)과 정확도(validation_accuracy)를 매 에포크마다 계산하고 출력한다.

import torch
import torch.optim as optim  # 최적화 도구 모음
import torch.nn.functional as F  # 활성화 함수, 손실 함수 등
from torch.utils.data import DataLoader, TensorDataset  # 데이터 로딩 유틸리티
import numpy as np
from tqdm import tqdm  # tqdm 라이브러리 import
import numpy as np

# 조기 종료 관련 하이퍼파라미터
patience = 5          # 검증 손실이 5번 연속으로 개선되지 않으면 훈련 중단
min_delta = 0.0001    # 최소한 이 값 이상 손실이 감소해야 개선으로 인정

best_loss = np.Inf    # 초기 최저 손실값 설정
patience_counter = 0  # 손실 개선 실패 횟수 카운터

# 모델 인스턴스화를 위한 변수 설정
# 이전 실험들과 일관성을 유지하거나 새로운 하이퍼파라미터를 설정한다.
vocab_size = 10000 
word_vector_dim = 100  # Word2Vec 임베딩을 위해 100으로 설정 (랜덤 실험 시에는 16)
max_length = 41
hidden_dim = 128  # LSTM의 은닉 상태 차원
num_layers = 2   # LSTM 레이어 수
dropout = 0.5  # 드롭아웃 확률
bidirectional = True # LSTM 양방향 설정
filter_sizes = [2, 3, 4] # CNN 필터 크기

# Word2Vec 임베딩 행렬을 사용한다고 가정하고, 이를 생성
# (실제 Word2Vec 모델 로드 및 임베딩 행렬 생성 코드는 이전에 수행되었어야 함)
# embedding_matrix = ...

# 모델 인스턴스화
model_lstm_cnn_word2vec = SentenceClassifier_LstmCnn(
    embedding_matrix=embedding_matrix, 
    hidden_dim=hidden_dim, 
    num_layers=num_layers, 
    dropout=dropout, 
    bidirectional=bidirectional, 
    filter_sizes=filter_sizes, 
    max_length=max_length
)

# 1. 모델 임베딩 초기화 (Word2Vec 사용 시)
# 모델의 임베딩 레이어를 사전 학습된 Word2Vec 벡터로 덮어씌운다.
# 랜덤 임베딩을 사용하는 실험에서는 이 줄을 주석 처리해야 한다.
# model_lstm_cnn_word2vec.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))

# 2. 옵티마이저와 손실 함수 설정
# Adam 최적화 도구와 이진 분류에 적합한 BCELoss 손실 함수를 사용한다.
optimizer = optim.Adam(model_lstm_cnn_word2vec.parameters())
loss_fn = torch.nn.BCELoss()

# 3. 데이터 로더 준비
# 훈련 및 검증 데이터를 텐서로 변환하고 데이터셋, 데이터로더를 생성한다.
partial_X_train_tensor = partial_X_train.detach().clone().to(torch.long)
partial_y_train_tensor = torch.tensor(partial_y_train, dtype=torch.float)

X_val_tensor = X_val.detach().clone().to(torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.float)

train_dataset = TensorDataset(partial_X_train_tensor, partial_y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)

epochs = 30  # 훈련 반복 횟수 설정
train_losses = []  # 에포크별 훈련 손실을 저장할 리스트
val_losses = []    # 에포크별 검증 손실을 저장할 리스트
train_accs = []    # 에포크별 훈련 정확도를 저장할 리스트
val_accs = []      # 에포크별 검증 정확도를 저장할 리스트

# 4. 모델 훈련 및 검증 루프
for epoch in range(epochs):
    # 훈련 단계 시작
    model_lstm_cnn_word2vec.train()  # 모델을 훈련 모드로 설정
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]"):
        optimizer.zero_grad()  # 이전 스텝에서 계산된 기울기 초기화
        outputs = model_lstm_cnn_word2vec(inputs)  # 모델에 입력 데이터를 넣어 예측값 계산
        loss = loss_fn(outputs.squeeze(), labels)  # 예측값과 실제 라벨 간의 손실 계산
        loss.backward()  # 손실에 대한 기울기 역전파
        optimizer.step()  # 모델의 파라미터 업데이트

        running_loss += loss.item()
        predicted = (outputs.squeeze() > 0.5).float()  # 예측값이 0.5보다 크면 긍정(1), 아니면 부정(0)으로 판단
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_losses.append(running_loss / len(train_loader))
    train_accs.append(correct / total)

    # 검증 단계 시작
    model_lstm_cnn_word2vec.eval()  # 모델을 평가 모드로 설정
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():  # 기울기 계산 비활성화 (메모리 절약, 속도 향상)
        for inputs, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Validation]"):
            outputs = model_lstm_cnn_word2vec(inputs)
            loss = loss_fn(outputs.squeeze(), labels)
            val_loss += loss.item()
            predicted = (outputs.squeeze() > 0.5).float()
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_losses.append(val_loss / len(val_loader))
    val_accs.append(val_correct / val_total)
    
    val_loss = val_losses[-1] # 현재 에포크의 검증 손실

    # 손실 개선 여부 확인
    if val_loss < best_loss - min_delta:
        best_loss = val_loss
        patience_counter = 0
        # 최적의 모델 가중치 저장
        torch.save(model_lstm_cnn_word2vec.state_dict(), 'model_lstm_cnn_word2vec.pth')
        print(f"Validation loss improved. Saving model weights...")
    else:
        patience_counter += 1
        print(f"Validation loss did not improve. Patience counter: {patience_counter}/{patience}")

    # 조기 종료 조건 확인
    if patience_counter >= patience:
        print(f"Early stopping triggered after {epoch+1} epochs.")
        break
    # 에포크별 결과 출력ㄴ
    print(f"Epoch {epoch+1}/{epochs} - "
          f"T_Loss: {train_losses[-1]:.4f}, T_Acc: {train_accs[-1]:.4f} - "
          f"V_Loss: {val_losses[-1]:.4f}, V_Acc: {val_accs[-1]:.4f}")

In [ ]:
# gensim 형식으로 저장할 파일 경로
word2vec_file_path = 'model_lstm_cnn_word2vec.txt'

# 모델의 임베딩 가중치와 단어-인덱스 매핑 준비
# 1. 모델 변수명을 'model_lstm_cnn_word2vec'로 수정
vectors = model_lstm_cnn_word2vec.embedding.weight.detach().cpu().numpy()

# 2. 모델 훈련 시 사용했던 실제 'word_to_index' 딕셔너리를 활용하여
#    'index_to_word' 딕셔너리를 생성합니다.
#    (이 부분은 실제 훈련 코드에 있는 'word_to_index' 변수명으로 교체해야 합니다.)
#    예시: word_to_index = {'<pad>': 0, '<unk>': 1, '영화': 2, ...}
#    - word_to_index 변수가 없으면 이 코드는 에러를 발생시킵니다.
index_to_word = {index: word for word, index in word_to_index.items()}


# 파일에 쓰기
with open(word2vec_file_path, 'w') as f:
    # 1. 파일의 첫 줄에 '전체 단어 수', '벡터 차원'을 기록합니다.
    #    gensim이 이 정보를 보고 파일 형식을 판단합니다.
    vocab_size, word_vector_dim = vectors.shape
    f.write(f'{vocab_size} {word_vector_dim}\n')
    
    # 2. 각 단어의 벡터 값을 공백으로 구분하여 한 줄씩 기록합니다.
    for i in range(vocab_size):
        # 3. index_to_word.get(i, '<unk>')를 사용하여 없는 인덱스도 안전하게 처리
        word = index_to_word.get(i, '<unk>')
        vector_str = ' '.join(map(str, vectors[i, :]))
        f.write(f'{word} {vector_str}\n')

print(f"'{word2vec_file_path}' 파일에 임베딩 가중치가 저장되었습니다.")

In [ ]:
test_dataset = TensorDataset(X_test_tensor, torch.tensor(y_test, dtype=torch.float32))
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

model_lstm_cnn_word2vec.load_state_dict(torch.load('model_lstm_cnn_word2vec.pth'))
model_lstm_cnn_word2vec.eval()
test_loss = 0.0
test_correct = 0
test_total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model_lstm_cnn_word2vec(inputs)
        loss = loss_fn(outputs.squeeze(), labels)

        test_loss += loss.item()
        predicted = (outputs.squeeze() > 0.5).float()
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)

print(f"Test Loss: {test_loss / len(test_loader):.4f}, Test Accuracy: {test_correct / test_total:.4f}")

In [ ]:
import matplotlib.pyplot as plt

epochs_range = range(1, len(train_losses) + 1)

# "bo"는 "파란색 점"입니다
plt.plot(epochs_range, train_losses, 'bo', label='Training loss')
# b는 "파란 실선"입니다
plt.plot(epochs_range, val_losses, 'b', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf()   # 그림을 초기화합니다

plt.plot(epochs_range, train_accs, 'bo', label='Training accuracy')
plt.plot(epochs_range, val_accs, 'b', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# 6종 임배딩 비교
#1. 코드 블록 제목: 6가지 모델 임베딩 파일 유사 단어 비교 (개선)
# gensim과 OS 관련 라이브러리 임포트
import os
from gensim.models.keyedvectors import Word2VecKeyedVectors

# 2. 파일 경로 및 설정
# 역할: 임베딩 파일들이 저장된 기본 경로를 설정
# 목적: 모든 파일 경로를 간편하게 관리하기 위함입니다.
base_path = os.path.join(os.getenv('HOME'), 'work', 'workplace', 'AIFFEL_quest_rs', 'Exploration', 'Quest05') + os.sep
# 역할: 비교할 6가지 임베딩 파일 목록을 리스트로 정의
# 목적: 반복문을 통해 각 파일을 순차적으로 처리하기 위함입니다.
embedding_files = [
    'model_lstm_random.txt',
    'model_lstm_word2vec.txt',
    'model_cnn_random.txt',
    'model_cnn_word2vec.txt',
    'model_lstm_cnn_random.txt',
    'model_lstm_cnn_word2vec.txt'
]
# 역할: 유사 단어를 찾을 대상 단어 목록을 정의
# 목적: 각 모델에서 동일한 단어들에 대한 유사 단어를 비교하기 위함입니다.
target_words = ['재미', '취향', '추천', '영화', '배우', '음악']

# 3. 임베딩 파일 로드
# 역할: 로드된 gensim 모델들을 저장할 딕셔너리 초기화
# 목적: 파일명(키)과 gensim 모델 객체(값)를 연결하여 편리하게 사용하기 위함입니다.
word_vectors_models = {}
print("--- 임베딩 파일 로드 시작 ---")
# 역할: embedding_files 리스트를 순회
# 목적: 6개의 파일을 하나씩 로드하고 딕셔너리에 저장합니다.
for filename in embedding_files:
    file_path = base_path + filename
    try:
        # 역할: gensim load_word2vec_format 함수를 사용해 파일을 로드
        # 목적: 텍스트 형식으로 저장된 임베딩 파일을 gensim 객체로 변환합니다.
        word_vectors = Word2VecKeyedVectors.load_word2vec_format(file_path, binary=False)
        word_vectors_models[filename] = word_vectors
        print(f"'{filename}' 로드 완료.")
    except FileNotFoundError:
        # 역할: 파일이 존재하지 않을 경우 에러를 출력하고 다음 파일로 넘어감
        # 목적: 파일 경로 오류로 인해 프로그램이 중단되는 것을 방지합니다.
        print(f"오류: '{filename}' 파일을 찾을 수 없습니다. 경로를 확인해주세요.")

# 4. 유사 단어 비교 및 결과 출력
print("\n--- 유사 단어 비교 시작 ---")
# 역할: target_words 리스트를 순회
# 목적: 각 단어에 대해 모든 모델에서 유사 단어를 찾습니다.
for word in target_words:
    print(f"\n==================== 단어: '{word}' ====================")
    # 역할: word_vectors_models 딕셔너리를 순회
    # 목적: 모든 로드된 모델에 대해 유사 단어를 찾습니다.
    for model_name, model in word_vectors_models.items():
        # 역할: 현재 단어가 모델의 어휘 사전에 있는지 확인
        # 목적: 사전에 없는 단어로 유사 단어를 찾으려 할 경우 에러가 나는 것을 방지합니다.
        if word in model:
            # 역할: gensim의 similar_by_word 함수를 사용해 유사 단어를 찾아 출력
            # 목적: 각 모델이 생각하는 'word'와 가장 비슷한 단어들을 보여줍니다.
            print(f"[{model_name}] 유사 단어:")
            similar_words = model.similar_by_word(word, topn=5)
            # 역할: 찾은 유사 단어를 보기 좋게 출력
            # 목적: 각 결과가 어떤 단어인지, 유사도 점수는 얼마인지 명확하게 표시합니다.
            for similar_word, score in similar_words:
                print(f"  - {similar_word} (유사도: {score:.4f})")
        else:
            # 역할: 단어가 사전에 없을 경우 메시지 출력
            # 목적: 해당 모델이 이 단어를 학습하지 못했음을 알려줍니다.
            print(f"[{model_name}] 사전에 '{word}' 단어가 없습니다.")

In [ ]:
# 0. 라이브러리 및 모델 클래스 임포트
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 모델 클래스 정의
class SentimentModel(nn.Module):
    def __init__(self, vocab_size, word_vector_dim):
        super(SentimentModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, word_vector_dim)
        self.lstm = nn.LSTM(word_vector_dim, 8, batch_first=True)
        self.fc1 = nn.Linear(8, 8)
        self.fc2 = nn.Linear(8, 1)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

class SentenceClassifier_Cnn(nn.Module):
    def __init__(self, embedding_matrix, filter_sizes, max_length, dropout=0.5):
        super().__init__()
        self.embedding = nn.Embedding(
            num_embeddings=embedding_matrix.shape[0],
            embedding_dim=embedding_matrix.shape[1]
        )
        self.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
        embedding_dim = self.embedding.weight.shape[1]

        conv = []
        for size in filter_sizes:
            conv.append(
                nn.Sequential(
                    nn.Conv1d(
                        in_channels=embedding_dim,
                        out_channels=1,
                        kernel_size=size
                    ),
                    nn.ReLU(),
                    nn.MaxPool1d(kernel_size=max_length-size-1),
                )
            )
        self.conv_filters = nn.ModuleList(conv)
        output_size = len(filter_sizes)
        self.pre_classifier = nn.Linear(output_size, output_size)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(output_size, 1)

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        embeddings = embeddings.permute(0, 2, 1)
        conv_outputs = [conv(embeddings) for conv in self.conv_filters]
        concat_outputs = torch.cat([conv.squeeze(-1) for conv in conv_outputs], dim=1)
        logits = self.pre_classifier(concat_outputs)
        logits = self.dropout(logits)
        logits = self.classifier(logits)
        return torch.sigmoid(logits)

class SentenceClassifier_LstmCnn(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, num_layers, dropout, bidirectional, filter_sizes, max_length):
        super().__init__()
        self.embedding = nn.Embedding(
            num_embeddings=embedding_matrix.shape[0],
            embedding_dim=embedding_matrix.shape[1]
        )
        self.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
        embedding_dim = self.embedding.weight.shape[1]
        
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            bidirectional=bidirectional,
            dropout=dropout,
            batch_first=True
        )
        lstm_output_dim = hidden_dim * 2 if bidirectional else hidden_dim
        
        conv = []
        for size in filter_sizes:
            conv.append(
                nn.Sequential(
                    nn.Conv1d(
                        in_channels=lstm_output_dim,
                        out_channels=1,
                        kernel_size=size
                    ),
                    nn.ReLU(),
                    nn.MaxPool1d(kernel_size=max_length-size-1),
                )
            )
        self.conv_filters = nn.ModuleList(conv)
        output_size = len(filter_sizes)
        self.pre_classifier = nn.Linear(output_size, output_size)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(output_size, 1)
        
    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        lstm_output, _ = self.lstm(embeddings)
        lstm_output = lstm_output.permute(0, 2, 1)
        conv_outputs = [conv(lstm_output) for conv in self.conv_filters]
        concat_outputs = torch.cat([conv.squeeze(-1) for conv in conv_outputs], dim=1)
        logits = self.pre_classifier(concat_outputs)
        logits = self.dropout(logits)
        logits = self.classifier(logits)
        return torch.sigmoid(logits)


# 1. 모델 파일 경로 및 설정 정보
base_path = os.path.join(os.getenv('HOME'), 'work', 'workplace', 'AIFFEL_quest_rs', 'Exploration', 'Quest05') + os.sep
model_configs = {
    'lstm_random': {'path': base_path + 'model_lstm_random.pth', 'class': SentimentModel, 'params': {'vocab_size': 10000, 'word_vector_dim': 16}},
    'lstm_word2vec': {'path': base_path + 'model_lstm_word2vec.pth', 'class': SentimentModel, 'params': {'vocab_size': 10000, 'word_vector_dim': 100}},
    'cnn_random': {'path': base_path + 'model_cnn_random.pth', 'class': SentenceClassifier_Cnn, 'params': {'embedding_matrix': np.random.rand(10000, 16), 'filter_sizes': [2, 3, 4], 'max_length': 41, 'dropout': 0.5}},
    'cnn_word2vec': {'path': base_path + 'model_cnn_word2vec.pth', 'class': SentenceClassifier_Cnn, 'params': {'embedding_matrix': np.random.rand(10000, 100), 'filter_sizes': [2, 3, 4], 'max_length': 41, 'dropout': 0.5}},
    'lstm_cnn_random': {'path': base_path + 'model_lstm_cnn_random.pth', 'class': SentenceClassifier_LstmCnn, 'params': {'embedding_matrix': np.random.rand(10000, 16), 'hidden_dim': 128, 'num_layers': 2, 'dropout': 0.5, 'bidirectional': True, 'filter_sizes': [2, 3, 4], 'max_length': 41}},
    'lstm_cnn_word2vec': {'path': base_path + 'model_lstm_cnn_word2vec.pth', 'class': SentenceClassifier_LstmCnn, 'params': {'embedding_matrix': np.random.rand(10000, 100), 'hidden_dim': 128, 'num_layers': 2, 'dropout': 0.5, 'bidirectional': True, 'filter_sizes': [2, 3, 4], 'max_length': 41}}
}

embedding_weights = {}

# 2. 각 모델 로드 및 임베딩 가중치 추출
for model_name, config in model_configs.items():
    print(f"--- {model_name} 임베딩 가중치 추출 중 ---")
    
    try:
        # 모델 인스턴스화
        model = config['class'](**config['params'])
        
        # 저장된 가중치 로드
        model.load_state_dict(torch.load(config['path']))
        
        # 임베딩 가중치 추출 및 저장
        # .detach().cpu().numpy()를 사용하여 텐서를 넘파이 배열로 변환
        embedding_weights[model_name] = model.embedding.weight.data.detach().cpu().numpy()
        
        print(f"{model_name} 임베딩 가중치 추출 완료. Shape: {embedding_weights[model_name].shape}")

    except FileNotFoundError:
        print(f"오류: {config['path']} 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
    except Exception as e:
        print(f"오류: {model_name} 모델 로드 및 가중치 추출 실패 - {e}")

# 3. 임베딩 가중치 간의 유사도(코사인 유사도) 계산 및 비교
print("\n### 임베딩 가중치 간의 코사인 유사도 비교 ###")

# 모든 모델 쌍에 대한 유사도를 저장할 딕셔너리
similarity_matrix = np.zeros((len(embedding_weights), len(embedding_weights)))
model_names = list(embedding_weights.keys())

# 유사도 계산
for i, name1 in enumerate(model_names):
    for j, name2 in enumerate(model_names):
        # 크기가 다른 임베딩을 비교하기 위해 크기가 더 작은 임베딩에 맞춰 슬라이싱
        dim1 = embedding_weights[name1].shape[1]
        dim2 = embedding_weights[name2].shape[1]
        min_dim = min(dim1, dim2)
        
        vec1 = embedding_weights[name1][:, :min_dim]
        vec2 = embedding_weights[name2][:, :min_dim]

        # 코사인 유사도 계산
        # 1. 벡터를 단위 벡터로 정규화
        vec1_norm = vec1 / np.linalg.norm(vec1, axis=1, keepdims=True)
        vec2_norm = vec2 / np.linalg.norm(vec2, axis=1, keepdims=True)
        
        # 2. 내적 계산 (코사인 유사도)
        # 내적의 평균을 유사도로 사용
        similarity = np.mean(np.sum(vec1_norm * vec2_norm, axis=1))
        similarity_matrix[i, j] = similarity

# 4. 결과 시각화
similarity_df = pd.DataFrame(similarity_matrix, index=model_names, columns=model_names)
print(similarity_df)

plt.figure(figsize=(10, 8))
sns.heatmap(similarity_df, annot=True, cmap='viridis', fmt=".4f")
plt.title('Embedding Weight Cosine Similarity Matrix')
plt.show()

# 5. 상세 분석
print("\n### 분석 결과 ###")
print("- 각 행과 열은 모델의 임베딩 가중치를 나타냅니다.")
print("- 히트맵의 값은 두 임베딩 가중치 벡터 간의 평균 코사인 유사도를 의미합니다.")
print("- 값이 1에 가까울수록 두 임베딩 가중치가 매우 유사하다는 뜻입니다.")
print("- Word2Vec 모델들 간의 유사도, Random 모델들 간의 유사도를 비교해보세요.")
print("- 예를 들어, 'lstm_word2vec'과 'cnn_word2vec'의 유사도는 'lstm_random'과 'cnn_random'의 유사도와 어떻게 다른가요?")

In [ ]:
# 0. 자체 학습한 임베딩을 `gensim` 형식으로 저장 및 로드하는 코드
#    (이전에 수행했던 코드)
import os
import torch
from gensim.models.keyedvectors import Word2VecKeyedVectors
from gensim.models import Word2Vec # Word2Vec 모델 로드를 위해 추가
import numpy as np

# model, vocab_size, word_vector_dim, index_to_word 변수가 정의되었다고 가정
word2vec_file_path = os.path.join(os.getenv('HOME'), 'work', 'sentiment_classification', 'word2vec_emb', 'word2vec.txt')
with open(word2vec_file_path, 'w', encoding='utf-8') as f:
    f.write('{} {}\n'.format(vocab_size - 4, word_vector_dim))
    vectors = model.embedding.weight.detach().cpu().numpy()
    for i in range(4, vocab_size):
        f.write('{} {}\n'.format(index_to_word[i], ' '.join(map(str, vectors[i, :]))))

# 자체 학습 임베딩 로드
word_vectors_self = Word2VecKeyedVectors.load_word2vec_format(word2vec_file_path, binary=False)

# 1. 사전 학습된 한국어 Word2Vec 임베딩 로드
#    (프로젝트 요구사항에 따라 word2vec_ko.model 사용)
korean_word2vec_path = os.path.join(os.getenv('HOME'), 'work', 'sentiment_classification', 'data', 'word2vec_ko.model')
word_vectors_ko = Word2Vec.load(korean_word2vec_path).wv

# 2. 특정 단어에 대한 유사 단어 비교 분석
def compare_similar_words(word, topn=10):
    print(f"================== '{word}' 유사 단어 비교 ==================")
    
    # 자체 학습 임베딩 결과
    # word_vectors_self의 key_to_index는 단어를 기준으로 확인
    if word in word_vectors_self.key_to_index:
        print("\n[자체 학습 임베딩 결과]")
        # gensim에서 유사 단어 찾기
        for similar_word, score in word_vectors_self.similar_by_word(word, topn=topn):
            print(f"{similar_word}: {score:.4f}")
    else:
        print(f"\n[자체 학습 임베딩] '{word}' 단어가 어휘 사전에 없습니다.")

    # 사전 학습 임베딩 결과
    # word_vectors_ko의 key_to_index는 단어를 기준으로 확인
    if word in word_vectors_ko.key_to_index:
        print("\n[사전 학습 임베딩 (word2vec_ko.model) 결과]")
        # gensim에서 유사 단어 찾기
        for similar_word, score in word_vectors_ko.similar_by_word(word, topn=topn):
            print(f"{similar_word}: {score:.4f}")
    else:
        print(f"\n[사전 학습 임베딩] '{word}' 단어가 어휘 사전에 없습니다.")

# 3. 비교 실행 (한국어 단어 사용)
compare_similar_words("재미", topn=5)
compare_similar_words("취향", topn=5)
compare_similar_words("추천", topn=5)

In [ ]:
# 0. 라이브러리 및 모델 클래스 임포트
# 필요한 라이브러리를 임포트합니다.
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
from gensim.models.keyedvectors import Word2VecKeyedVectors
from collections import OrderedDict
from typing import Dict, Any

# 모델 클래스 정의
# 1_1과 1_2 실험에서 사용한 LSTM 모델 클래스입니다.
class SentimentModel(nn.Module):
    def __init__(self, vocab_size, word_vector_dim):
        super(SentimentModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, word_vector_dim)
        self.lstm = nn.LSTM(word_vector_dim, 8, batch_first=True)
        self.fc1 = nn.Linear(8, 8)
        self.fc2 = nn.Linear(8, 1)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# 2_1과 2_2 실험에서 사용한 CNN 모델 클래스입니다.
class SentenceClassifier_Cnn(nn.Module):
    def __init__(self, embedding_matrix, filter_sizes, max_length, dropout=0.5):
        super().__init__()
        self.embedding = nn.Embedding(
            num_embeddings=embedding_matrix.shape[0],
            embedding_dim=embedding_matrix.shape[1]
        )
        self.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
        embedding_dim = self.embedding.weight.shape[1]

        conv = []
        for size in filter_sizes:
            # Conv1d의 out_channels를 128로 설정하여 더 많은 특징을 추출하도록 수정
            conv.append(
                nn.Sequential(
                    nn.Conv1d(
                        in_channels=embedding_dim,
                        out_channels=128, 
                        kernel_size=size
                    ),
                    nn.ReLU(),
                    # MaxPool1d를 AdaptiveMaxPool1d(1)로 변경하여 유연하게 풀링하도록 수정
                    nn.AdaptiveMaxPool1d(1),
                )
            )
        self.conv_filters = nn.ModuleList(conv)
        # out_channels=128로 변경했으므로, output_size도 (필터 개수 * 필터 수)로 변경
        output_size = len(filter_sizes) * 128
        self.pre_classifier = nn.Linear(output_size, output_size)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(output_size, 1)

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        embeddings = embeddings.permute(0, 2, 1)
        conv_outputs = [conv(embeddings) for conv in self.conv_filters]
        concat_outputs = torch.cat([conv.squeeze(-1) for conv in conv_outputs], dim=1)
        logits = self.pre_classifier(concat_outputs)
        logits = self.dropout(logits)
        logits = self.classifier(logits)
        return torch.sigmoid(logits)

# 3_1과 3_2 실험에서 사용한 LSTM-CNN 모델 클래스입니다.
class SentenceClassifier_LstmCnn(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, num_layers, dropout, bidirectional, filter_sizes, max_length):
        super().__init__()
        self.embedding = nn.Embedding(
            num_embeddings=embedding_matrix.shape[0],
            embedding_dim=embedding_matrix.shape[1]
        )
        self.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
        embedding_dim = self.embedding.weight.shape[1]
        
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            bidirectional=bidirectional,
            dropout=dropout,
            batch_first=True
        )
        lstm_output_dim = hidden_dim * 2 if bidirectional else hidden_dim
        
        conv = []
        for size in filter_sizes:
            # Conv1d의 out_channels를 128로 설정하여 더 많은 특징을 추출하도록 수정
            conv.append(
                nn.Sequential(
                    nn.Conv1d(
                        in_channels=lstm_output_dim,
                        out_channels=128,
                        kernel_size=size
                    ),
                    nn.ReLU(),
                    # MaxPool1d를 AdaptiveMaxPool1d(1)로 변경하여 유연하게 풀링하도록 수정
                    nn.AdaptiveMaxPool1d(1),
                )
            )
        self.conv_filters = nn.ModuleList(conv)
        # out_channels=128로 변경했으므로, output_size도 (필터 개수 * 필터 수)로 변경
        output_size = len(filter_sizes) * 128
        self.pre_classifier = nn.Linear(output_size, output_size)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(output_size, 1)
        
    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        lstm_output, _ = self.lstm(embeddings)
        lstm_output = lstm_output.permute(0, 2, 1)
        conv_outputs = [conv(lstm_output) for conv in self.conv_filters]
        concat_outputs = torch.cat([conv.squeeze(-1) for conv in conv_outputs], dim=1)
        logits = self.pre_classifier(concat_outputs)
        logits = self.dropout(logits)
        logits = self.classifier(logits)
        return torch.sigmoid(logits)

# 1. 모델 파일 경로 및 설정 정보
# 이전에 훈련하여 저장한 6개의 모델 파일 경로와 설정을 정의합니다.
base_path = os.path.join(os.getenv('HOME'), 'work', 'workplace', 'AIFFEL_quest_rs', 'Exploration', 'Quest05') + os.sep
model_configs = OrderedDict([
    ('lstm_random', {'path': base_path + 'model_lstm_random.pth', 'class': SentimentModel, 'params': {'vocab_size': 10000, 'word_vector_dim': 16}}),
    ('lstm_word2vec', {'path': base_path + 'model_lstm_word2vec.pth', 'class': SentimentModel, 'params': {'vocab_size': 10000, 'word_vector_dim': 100}}),
    ('cnn_random', {'path': base_path + 'model_cnn_random.pth', 'class': SentenceClassifier_Cnn, 'params': {'embedding_matrix': np.random.rand(10000, 16), 'filter_sizes': [2, 3, 4], 'max_length': 41, 'dropout': 0.5}}),
    ('cnn_word2vec', {'path': base_path + 'model_cnn_word2vec.pth', 'class': SentenceClassifier_Cnn, 'params': {'embedding_matrix': np.random.rand(10000, 100), 'filter_sizes': [2, 3, 4], 'max_length': 41, 'dropout': 0.5}}),
    ('lstm_cnn_random', {'path': base_path + 'model_lstm_cnn_random.pth', 'class': SentenceClassifier_LstmCnn, 'params': {'embedding_matrix': np.random.rand(10000, 16), 'hidden_dim': 128, 'num_layers': 2, 'dropout': 0.5, 'bidirectional': True, 'filter_sizes': [2, 3, 4], 'max_length': 41}}),
    ('lstm_cnn_word2vec', {'path': base_path + 'model_lstm_cnn_word2vec.pth', 'class': SentenceClassifier_LstmCnn, 'params': {'embedding_matrix': np.random.rand(10000, 100), 'hidden_dim': 128, 'num_layers': 2, 'dropout': 0.5, 'bidirectional': True, 'filter_sizes': [2, 3, 4], 'max_length': 41}}),
])

# 2. 단어-인덱스 매핑 정보 생성 (더미 데이터)
# 실제 프로젝트에서는 이전 단계에서 저장된 `index_to_word`를 사용해야 합니다.
# 여기서는 예시를 위해 간단하게 생성합니다.
index_to_word_path = os.path.join(os.getenv('HOME'), 'work', 'workplace', 'AIFFEL_quest_rs', 'Exploration', 'Quest05', 'index_to_word.json')
try:
    import json
    with open(index_to_word_path, 'r', encoding='utf-8') as f:
        index_to_word = json.load(f)
except FileNotFoundError:
    print(f"경고: {index_to_word_path} 파일을 찾을 수 없습니다. 더미 데이터를 생성합니다.")
    index_to_word = {i: f"단어{i}" for i in range(10000)}
    index_to_word[0] = '<pad>'
    index_to_word[1] = '<unk>'
    index_to_word[2] = '재미'
    index_to_word[3] = '취향'
    index_to_word[4] = '추천'
    index_to_word[5] = '영화'
    index_to_word[6] = '배우'

# gensim 형식으로 변환된 임베딩을 저장할 딕셔너리
gensim_embeddings = {}

# 3. 각 모델 로드 및 임베딩 가중치 추출 후 gensim 형식으로 변환
for model_name, config in model_configs.items():
    print(f"--- '{model_name}' 임베딩 가중치 추출 및 gensim 변환 중 ---")
    
    try:
        # 모델 인스턴스화
        model = config['class'](**config['params'])
        
        # 저장된 가중치 로드
        model.load_state_dict(torch.load(config['path']))
        
        # 임베딩 가중치 추출
        embedding_matrix = model.embedding.weight.data.detach().cpu().numpy()
        
        # gensim KeyedVectors 객체 생성
        word_vectors = Word2VecKeyedVectors(vector_size=embedding_matrix.shape[1])
        word_vectors.add_vectors(
            list(index_to_word.values()), # 단어 리스트
            embedding_matrix            # 임베딩 행렬
        )
        gensim_embeddings[model_name] = word_vectors
        
        print(f"'{model_name}' 임베딩 변환 완료. 벡터 크기: {embedding_matrix.shape[1]}")

    except FileNotFoundError:
        print(f"오류: {config['path']} 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
    except Exception as e:
        print(f"오류: '{model_name}' 모델 로드 및 변환 실패 - {e}")

# 4. 특정 단어에 대한 유사 단어 비교 분석 함수
def compare_similar_words_for_all_models(word: str, topn: int = 5):
    """
    6가지 모델의 임베딩을 사용하여 특정 단어와 유사한 단어를 비교합니다.
    """
    print(f"\n================== '{word}' 유사 단어 비교 ==================")
    for model_name, word_vectors in gensim_embeddings.items():
        print(f"\n--- {model_name} 결과 (차원: {word_vectors.vector_size}) ---")
        if word in word_vectors.key_to_index:
            try:
                # gensim에서 유사 단어 찾기
                for similar_word, score in word_vectors.similar_by_word(word, topn=topn):
                    print(f"  - {similar_word}: {score:.4f}")
            except Exception as e:
                print(f"  - 유사 단어 찾기 실패: {e}")
        else:
            print(f"  - '{word}' 단어가 어휘 사전에 없습니다.")

# 5. 비교 실행 (한국어 단어 사용)
# 6가지 모델에 대한 유사 단어 비교를 수행합니다.
compare_similar_words_for_all_models("재미", topn=5)
compare_similar_words_for_all_models("취향", topn=5)
compare_similar_words_for_all_models("추천", topn=5)
compare_similar_words_for_all_models("영화", topn=5)
compare_similar_words_for_all_models("배우", topn=5)


In [ ]:
# 0. 코드의 목적: PyTorch 모델 파일에서 임베딩 벡터를 추출하여 gensim 방식으로 확인

# 1. 필요한 라이브러리 임포트
# PyTorch 모델을 다루기 위한 라이브러리
import torch
import torch.nn as nn
# 넘파이 배열로 변환하기 위한 라이브러리
import numpy as np
# gensim KeyedVectors 객체를 생성하기 위한 라이브러리
from gensim.models.keyedvectors import Word2VecKeyedVectors
# 파일 경로 설정을 위한 라이브러리
import os

# 2. PyTorch 모델 클래스 정의 (lstm_random 모델용)
class SentimentModel(nn.Module):
    # 모델의 생성자 함수입니다.
    def __init__(self, vocab_size, word_vector_dim):
        super(SentimentModel, self).__init__()
        # nn.Embedding 레이어를 정의합니다. 이 레이어에 단어 임베딩이 저장됩니다.
        self.embedding = nn.Embedding(vocab_size, word_vector_dim)
        # LSTM, Linear 등 다른 레이어들도 정의하지만, 여기서는 임베딩만 사용합니다.
        self.lstm = nn.LSTM(word_vector_dim, 8, batch_first=True)
        self.fc1 = nn.Linear(8, 8)
        self.fc2 = nn.Linear(8, 1)

    # 모델의 순전파(forward) 함수입니다.
    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# 3. 모델 파일 경로 및 설정 정의
# 사용자의 환경에 맞게 경로를 설정합니다.
base_path = os.path.join(os.getenv('HOME'), 'work', 'workplace', 'AIFFEL_quest_rs', 'Exploration', 'Quest05') + os.sep
model_file_path = base_path + 'model_lstm_random.pth'
vocab_size = 10000
word_vector_dim = 16

# 4. 모델 로딩 및 임베딩 가중치 추출
# 모델 클래스의 인스턴스를 생성합니다.
model = SentimentModel(vocab_size, word_vector_dim)
# 저장된 가중치를 모델에 로드합니다.
model.load_state_dict(torch.load(model_file_path))
# 모델의 임베딩 레이어에서 가중치(weight)를 추출하고, 넘파이 배열로 변환합니다.
embedding_matrix = model.embedding.weight.data.detach().cpu().numpy()

# 5. 간단한 단어-인덱스 매핑 정의
# 실제 단어를 인덱스와 매핑하는 딕셔너리를 간단히 정의합니다.
index_to_word = {
    0: '<pad>', 1: '<unk>', 2: '재미', 3: '취향', 4: '추천',
    5: '영화', 6: '배우', 7: '음악', 8: '친구'
}

# 6. gensim KeyedVectors 객체 생성
# 추출한 가중치와 단어 매핑 정보를 이용하여 KeyedVectors 객체를 만듭니다.
word_vectors = Word2VecKeyedVectors(vector_size=word_vector_dim)
word_vectors.add_vectors(
    list(index_to_word.values()),   # 단어 리스트
    embedding_matrix[:len(index_to_word)]  # 해당 단어에 맞는 임베딩 벡터만 사용
)

# 7. 원하는 단어의 벡터 확인
# '재미'라는 단어의 임베딩 벡터를 출력합니다.
print("================ '재미' 단어의 임베딩 벡터 확인 ================")
if '재미' in word_vectors:
    # KeyedVectors 객체에서 '재미' 단어의 벡터를 가져옵니다.
    vector = word_vectors['재미']
    # 벡터의 처음 5개 요소를 출력하여 정상적으로 로드되었음을 확인합니다.
    print(f"['재미'] 단어의 벡터 (일부): {vector[:5]}")
    # '재미'와 가장 유사한 단어들을 출력합니다.
    print(f"\n['재미']와 유사한 단어: {word_vectors.similar_by_word('재미', topn=3)}")
else:
    print("'재미' 단어가 어휘 사전에 없습니다.")
